### Study Correlation Plan

For the purpose of getting the HRV data, we will use the library Neurokit2 to handle the proceess to get the data short window and the full one.

### Flow of the Study

- Takes the Windowed version of the data (30 seconds, 1 minute and 2 minute)
- Calculate the HRV Metrics / Features
- Take the signal of the full length
- Take the study correlation

### HRV Metrics that we're going to use

| **Domain**     | **HRV Feature** | **Unit** | **Description**                                                                 |
|----------------|------------------|----------|----------------------------------------------------------------------------------|
| **Time**       | MeanNN           | ms       | Mean NN interval (Time it takes between each peak to peak) in milis              |
|                | SDNN             | ms       | Standard deviation of the RR intervals                                           |
|                | pNN50            | %        | NN50 count divided by the total number of all RR intervals                       |
|                | RMSSD            | ms       | Root mean square of successive RR interval differences                           |
|                | MeanHR           | bpm      | Mean heart rate                                                                  |
| **Frequency**  | LF               | ms²      | Power of low frequency band (0.04–0.15 Hz)                                       |
|                | HF               | ms²      | Power of high frequency band (0.15–0.4 Hz)                                       |
|                | LF/HF            | -        | Ratio of LF to HF                                                                |
| **Non-linear** | CSI              | -        | Cardiac sympathetic index                                                        |
|                | CVI              | -        | Cardiac vagal index                                                              |
|                | SD1              | -        | Standard deviation of Poincaré plot projection on the line perpendicular to line y=x |
|                | SD2              | -        | Standard deviation of Poincaré plot projection on the line y=x                  |


### Setup Requirements

In [1]:
# UST HRV and Normal HRV Correlation Analysis for Stress Detection
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os
from glob import glob
import warnings
import neurokit2 as nk
warnings.filterwarnings('ignore')

# Set plot style
plt.style.use('ggplot')
sns.set(font_scale=1.2)
sns.set_style("whitegrid")

In [ ]:
import scipy 

def preprocess_ppg(signal, fs = 35):
    """ Computes the Preprocessed PPG Signal, this steps include the following:
        1. Moving Average Smoothing
        2. Bandpass Filtering
        
        Parameters:
        ----------
        signal (numpy array): 
            The PPG Signal to be preprocessed
        fs (float): 
            The Sampling Frequency of the Signal
            
        Returns:
        --------
        numpy array: 
            The Preprocessed PPG Signal
    
    """ 

    # 2. Bandpass filter to isolate the cardiac component (0.4-2.5 Hz)
    b_bp, a_bp = scipy.signal.butter(3, [0.7, 2.5], btype='band', fs=fs)
    filtered = scipy.signal.filtfilt(b_bp, a_bp, signal)
    

    return filtered

# 30 Seconds Plot Correlation

For 30 seconds window, the averaging purpose will be done under windowing each short rPPG segment with the **strides** of 15 seconds (means the different between each short window is 15 seconds).

The test will be done under certain scenario of the Task 1, Task 2 UBFC, Physio Rest 2 and Rest 6

In [ ]:
root_path = "UBFC-Phys"
subjects = ["s41", "s42", "s43", "s44","s45","s46","s47","s48","s49","s50","s51","s52", "s53","s54","s55","s56"]
tasks = ["T3"]

# Store ground truth and rPPG data
gt_data = {}
rppg_data = {
    'POS': {},
    'LGI': {},
    'OMIT': {},
    'GREEN': {},
    'CHROM': {}
}
# Expected sampling rates (adjust if different for your dataset)
sample_rate_gt = 64  # Hz
sample_rate_video = 35 # Hz


In [4]:
## Process for each subject and task
for subject in subjects:
    for task in tasks:
        subject_task_id = f"{subject}_{task}"

        # Load rPPG signals from different methods
        pos = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_POS_rppg.npy"))
        lgi = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_LGI_rppg.npy"))
        omit = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_OMIT_rppg.npy"))
        green = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_GREEN_rppg.npy"))
        chrom = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_CHROM_rppg.npy"))

        # Load ground truth BVP
        GT = pd.read_csv(os.path.join(root_path, subject, f"bvp_{subject}_{task}.csv")).values
        GT = GT.flatten()

        ## process rPPG signals
        rppg_data["POS"][subject_task_id] = preprocess_ppg(pos, fs=sample_rate_video)
        rppg_data["LGI"][subject_task_id] = preprocess_ppg(lgi, fs=sample_rate_video)
        rppg_data["OMIT"][subject_task_id] = preprocess_ppg(omit, fs=sample_rate_video)
        rppg_data["GREEN"][subject_task_id] = preprocess_ppg(green, fs=sample_rate_video)
        rppg_data["CHROM"][subject_task_id] = preprocess_ppg(chrom, fs=sample_rate_video)
        
        GT = preprocess_ppg(GT, fs=sample_rate_gt)
        gt_data[subject_task_id] = GT

print(f"Done Process the Signals")
    

Done Process the Signals


In [ ]:
"""
Steps to reproduce getting the short term of 30 seconds for each subject + averaging:
1. Loop through each subject.
2. For each short rppg segment (30 seconds), compute the hrv metrics with the neurokit2 package and store it.
3. Average the HRV metrics across all segments for each subject.
4. Compare the correlation between the averaged HRV metrics of the rPPG methods and the ground truth HRV metrics.
# Note: The above code is a preprocessing step. The next steps would involve calculating HRV metrics and performing correlation analysis.
""" 

## Iterate for each subject and compute HRV metrics
hrv_metrics = {
    'MeanNN': [],
    'SDNN': [],
    'RMSSD': [],
    'pNN50': [],
    'LF': [],
    'HF': [],
    'LF_HF': [],
    'PR' : [],
}

## Store the HRV metrics for each rPPG method for each subject
rppg_hrv_metrics = {
    method: {
        subject_id: {
            key: [] for key in hrv_metrics.keys()
        } for subject_id in rppg_data[method].keys()
    } for method in rppg_data.keys()
}

## Iterate through each subject and compute HRV for each segments
for rppg_method in rppg_data.keys():
    for subject_task_id, rppg_signal in rppg_data[rppg_method].items():
        print(f"Processing {subject_task_id} for {rppg_method}")

        ## Applied the window of 30 seconds with stride of 15 seconds
        segment_length = 30 * sample_rate_video
        stride_length = 15 * sample_rate_video
        
        ## Making the segments
        for start in range(0, len(rppg_signal) - segment_length + 1, stride_length):
            segment = rppg_signal[start:start + segment_length]
            ## If the segment is less than the segment length, we skip it
            if len(segment) < segment_length:
                continue

            ## Compute the HRV metrics using neurokit2
            signals, _ = nk.ppg_process(segment, sampling_rate=sample_rate_video)
            peaks, _ = nk.ppg_peaks(signals["PPG_Clean"], sampling_rate=sample_rate_video)

            ## Getting the HR and store it in the metrics
            rppg_hrv_metrics[rppg_method][subject_task_id]['PR'].append(signals['PPG_Rate'][0])

            # Getting the HRV Metrics
            ## Time Domain
            hrv_time = nk.hrv_time(peaks, sampling_rate=sample_rate_video)

            ## Add into the hrv_metrics dictionary
            rppg_hrv_metrics[rppg_method][subject_task_id]['MeanNN'].append(hrv_time['HRV_MeanNN'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['SDNN'].append(hrv_time['HRV_SDNN'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['RMSSD'].append(hrv_time['HRV_RMSSD'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['pNN50'].append(hrv_time['HRV_pNN50'])

            ## Frequency Domain
            hrv_freq = nk.hrv_frequency(peaks, sampling_rate=sample_rate_video, psd_method="welch")
            rppg_hrv_metrics[rppg_method][subject_task_id]['LF'].append(hrv_freq['HRV_LF'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['HF'].append(hrv_freq['HRV_HF'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['LF_HF'].append(hrv_freq['HRV_LFHF'])

            ## Non-Linear Domain
            # hrv_non_linear = nk.hrv_nonlinear(peaks, sampling_rate=sample_rate_video)
            # rppg_hrv_metrics[rppg_method][subject_task_id]['SD1'].append(hrv_non_linear['HRV_SD1'])
            # rppg_hrv_metrics[rppg_method][subject_task_id]['SD2'].append(hrv_non_linear['HRV_SD2'])

Processing s41_T3 for POS
Processing s42_T3 for POS
Processing s43_T3 for POS
Processing s44_T3 for POS
Processing s45_T3 for POS
Processing s46_T3 for POS
Processing s47_T3 for POS
Processing s48_T3 for POS
Processing s49_T3 for POS
Processing s50_T3 for POS
Processing s51_T3 for POS
Processing s52_T3 for POS
Processing s53_T3 for POS
Processing s54_T3 for POS
Processing s55_T3 for POS
Processing s56_T3 for POS
Processing s41_T3 for LGI
Processing s42_T3 for LGI
Processing s43_T3 for LGI
Processing s44_T3 for LGI
Processing s45_T3 for LGI
Processing s46_T3 for LGI
Processing s47_T3 for LGI
Processing s48_T3 for LGI
Processing s49_T3 for LGI
Processing s50_T3 for LGI
Processing s51_T3 for LGI
Processing s52_T3 for LGI
Processing s53_T3 for LGI
Processing s54_T3 for LGI
Processing s55_T3 for LGI
Processing s56_T3 for LGI
Processing s41_T3 for OMIT
Processing s42_T3 for OMIT
Processing s43_T3 for OMIT
Processing s44_T3 for OMIT
Processing s45_T3 for OMIT
Processing s46_T3 for OMIT
Proces

In [6]:
### Calculate the average HRV metrics for each segment for each subject per method

hrv_means = {}
for method in rppg_hrv_metrics:
    hrv_means[method] = {}

    for subject in rppg_hrv_metrics[method]:
        hrv_means[method][subject] = {}

        for metric, values in rppg_hrv_metrics[method][subject].items():
            if values:
                hrv_means[method][subject][metric] = np.mean(values)
            else:
                hrv_means[method][subject][metric] = np.nan

print(hrv_means)

{'POS': {'s41_T3': {'MeanNN': 685.0317834975585, 'SDNN': 219.68978424069397, 'RMSSD': 306.0298990329968, 'pNN50': 86.25098347757671, 'LF': 0.03656729562539674, 'HF': 0.08870333543958615, 'LF_HF': 0.41070431814114605, 'PR': 87.5888990054854}, 's42_T3': {'MeanNN': 724.780726519857, 'SDNN': 252.56196906637493, 'RMSSD': 345.6294329796904, 'pNN50': 86.38878907653611, 'LF': 0.054039582421163745, 'HF': 0.12504953823098267, 'LF_HF': 0.4632886940917657, 'PR': 82.82881115957058}, 's43_T3': {'MeanNN': 767.450814650414, 'SDNN': 261.8628363571079, 'RMSSD': 317.84711681678004, 'pNN50': 91.51211716632277, 'LF': 0.027518121616339025, 'HF': 0.053430699152060245, 'LF_HF': 0.4766045907676057, 'PR': 78.2321738638538}, 's44_T3': {'MeanNN': 872.1295906678821, 'SDNN': 246.6780704846748, 'RMSSD': 316.15141220827127, 'pNN50': 87.27711445402905, 'LF': 0.055609081258518045, 'HF': 0.10351774876043252, 'LF_HF': 0.5402525196045955, 'PR': 68.80216363041954}, 's45_T3': {'MeanNN': 805.0633549134695, 'SDNN': 265.909201

### Getting the GT HRV Metrics

In [7]:
# Compare the Correlation between the averaged HRV metrics of the rPPG methods and the ground truth HRV metrics

## Getting the ground truth HRV metrics

gt_hrv_metrics = {
    subject_id: {
        key: [] for key in hrv_metrics.keys()
    } for subject_id in gt_data.keys()
}

# Iterate through each subject and compute the full length HRV metrics for the ground truth
for subject_task_id, gt_signal in gt_data.items():
    print(f"Processing {subject_task_id} for ground truth")

    ## Compute the HRV metrics using neurokit2
    signals, _ = nk.ppg_process(gt_signal, sampling_rate=sample_rate_gt)
    peaks, _ = nk.ppg_peaks(signals["PPG_Clean"], sampling_rate=sample_rate_gt)

    ## Getting the HR and store it in the metrics
    gt_hrv_metrics[subject_task_id]['PR'] = signals['PPG_Rate'][0].item()
    
    # Getting the HRV Metrics

    ## Time Domain
    hrv_time = nk.hrv_time(peaks, sampling_rate=sample_rate_gt)

    ## Add into the hrv_metrics dictionary
    gt_hrv_metrics[subject_task_id]['MeanNN'] = (hrv_time['HRV_MeanNN'][0])
    gt_hrv_metrics[subject_task_id]['SDNN'] = (hrv_time['HRV_SDNN'][0])
    gt_hrv_metrics[subject_task_id]['RMSSD'] = (hrv_time['HRV_RMSSD'][0])
    gt_hrv_metrics[subject_task_id]['pNN50'] = (hrv_time['HRV_pNN50'][0])

    ## Frequency Domain
    hrv_freq = nk.hrv_frequency(peaks, sampling_rate=sample_rate_gt, psd_method="welch")
    gt_hrv_metrics[subject_task_id]['LF'] = (hrv_freq['HRV_LF'][0])
    gt_hrv_metrics[subject_task_id]['HF'] = (hrv_freq['HRV_HF'][0])
    gt_hrv_metrics[subject_task_id]['LF_HF'] = (hrv_freq['HRV_LFHF'][0])

    ## Non-Linear Domain
    # hrv_non_linear = nk.hrv_nonlinear(peaks, sampling_rate=sample_rate_gt)
    # gt_hrv_metrics[subject_task_id]['SD1'] = (hrv_non_linear['HRV_SD1'])
    # gt_hrv_metrics[subject_task_id]['SD2'] = (hrv_non_linear['HRV_SD2'])

print(gt_hrv_metrics)

Processing s41_T3 for ground truth
Processing s42_T3 for ground truth
Processing s43_T3 for ground truth
Processing s44_T3 for ground truth
Processing s45_T3 for ground truth
Processing s46_T3 for ground truth
Processing s47_T3 for ground truth
Processing s48_T3 for ground truth
Processing s49_T3 for ground truth
Processing s50_T3 for ground truth
Processing s51_T3 for ground truth
Processing s52_T3 for ground truth
Processing s53_T3 for ground truth
Processing s54_T3 for ground truth
Processing s55_T3 for ground truth
Processing s56_T3 for ground truth
{'s41_T3': {'MeanNN': 759.2690677966102, 'SDNN': 256.2243539979767, 'RMSSD': 323.64115992706513, 'pNN50': 78.8135593220339, 'LF': 0.04414474180694006, 'HF': 0.0713327946728041, 'LF_HF': 0.6188561938366115, 'PR': 79.02336937565399}, 's42_T3': {'MeanNN': 739.7985537190083, 'SDNN': 188.31094883796968, 'RMSSD': 236.80021181113136, 'pNN50': 73.55371900826447, 'LF': 0.031234568707740655, 'HF': 0.07134949016888528, 'LF_HF': 0.43776863203658467

### Since we already get the Metrics HRV value of the rPPG, let's compare it with the GT to see the correlation

In [8]:
def identify_outliers_iqr(data):
    """Identify outlier indices using the IQR method.
    
    Parameters:
    ----------
    data (list or numpy array): The data to check for outliers.
    
    Returns:
    --------
    numpy array: Boolean mask where True indicates outlier.
    """
    data = np.asarray(data)
    
    if len(data) == 0:
        return np.array([], dtype=bool)
    
    if len(data) == 1:
        return np.array([False])
    
    # Remove any NaN or infinite values before calculating percentiles
    clean_data = data[np.isfinite(data)]
    
    if len(clean_data) < 2:
        return np.array([False] * len(data))
    
    q1 = np.percentile(clean_data, 25)
    q3 = np.percentile(clean_data, 75)
    iqr = q3 - q1
    
    # Handle case where IQR is 0 (all values are the same)
    if iqr == 0:
        return np.array([False] * len(data))
    
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    outlier_mask = (data < lower_bound) | (data > upper_bound) | ~np.isfinite(data)
    return outlier_mask

# Compute correlation between rPPG methods and ground truth HRV metrics
correlation_results = {}
plot_data = {}  # Store clean data for plotting

for method in hrv_means.keys():
    correlation_results[method] = {}
    plot_data[method] = {}
    
    for metric in hrv_metrics.keys():
        # Collect paired data (subject_id, rppg_value, gt_value)
        paired_data = []
        
        for subject_id in hrv_means[method].keys():
            # Check if both rPPG and GT data exist for this subject and metric
            rppg_available = (subject_id in hrv_means[method] and 
                            metric in hrv_means[method][subject_id])
            gt_available = (subject_id in gt_hrv_metrics and 
                          metric in gt_hrv_metrics[subject_id])
            
            if rppg_available and gt_available:
                rppg_value = hrv_means[method][subject_id][metric]
                gt_value = gt_hrv_metrics[subject_id][metric]
                
                # Handle pandas Series if needed
                if isinstance(gt_value, pd.Series):
                    if not gt_value.empty:
                        gt_value = gt_value.iloc[0]
                    else:
                        continue
                
                # Handle numpy arrays - extract scalar value
                if isinstance(rppg_value, (np.ndarray, list)):
                    if len(rppg_value) > 0:
                        rppg_value = rppg_value[0] if hasattr(rppg_value, '__getitem__') else float(rppg_value)
                    else:
                        continue
                
                if isinstance(gt_value, (np.ndarray, list)):
                    if len(gt_value) > 0:
                        gt_value = gt_value[0] if hasattr(gt_value, '__getitem__') else float(gt_value)
                    else:
                        continue
                
                # Convert to float to ensure scalar values
                try:
                    rppg_value = float(rppg_value)
                    gt_value = float(gt_value)
                except (TypeError, ValueError):
                    print(f"Warning: Could not convert values to float for {subject_id} - {metric}")
                    print(f"  rPPG value type: {type(rppg_value)}, value: {rppg_value}")
                    print(f"  GT value type: {type(gt_value)}, value: {gt_value}")
                    continue
                
                # Check for valid values (now they're guaranteed to be scalars)
                if not np.isnan(rppg_value) and not np.isnan(gt_value):
                    paired_data.append((subject_id, rppg_value, gt_value))
        
        if len(paired_data) < 2:
            print(f"Insufficient data for {method} - {metric}: {len(paired_data)} subjects")
            continue
        
        # Extract values for outlier detection
        subject_ids = [item[0] for item in paired_data]
        rppg_values = np.array([item[1] for item in paired_data])
        gt_values = np.array([item[2] for item in paired_data])
        
        # Debug information
        print(f"Debug - {method} - {metric}:")
        print(f"  Total paired subjects: {len(paired_data)}")
        print(f"  rPPG values shape: {rppg_values.shape}")
        print(f"  GT values shape: {gt_values.shape}")
        print(f"  rPPG values: {rppg_values}")
        print(f"  GT values: {gt_values}")
        
        # Identify outliers in both datasets
        rppg_outliers = identify_outliers_iqr(rppg_values)
        gt_outliers = identify_outliers_iqr(gt_values)
        
        # Combine outlier masks (remove if outlier in either dataset)
        combined_outliers = rppg_outliers | gt_outliers
        
        # Keep only non-outlier subjects
        clean_mask = ~combined_outliers
        clean_rppg_values = rppg_values[clean_mask]
        clean_gt_values = gt_values[clean_mask]
        clean_subject_ids = [subject_ids[i] for i in range(len(subject_ids)) if clean_mask[i]]
        
        print(f"{method} - {metric}: Removed {np.sum(combined_outliers)} outliers, "
              f"kept {len(clean_rppg_values)} subjects")
        
        # Store clean data for plotting
        plot_data[method][metric] = {
            'rppg_values': clean_rppg_values,
            'gt_values': clean_gt_values,
            'subject_ids': clean_subject_ids
        }
        
        # Calculate correlation on clean data
        if len(clean_rppg_values) > 1:
            correlation, p_value = stats.pearsonr(clean_rppg_values, clean_gt_values)
            correlation_results[method][metric] = {
                'correlation': correlation,
                'p_value': p_value,
                'n_subjects': len(clean_rppg_values),
                'removed_subjects': np.sum(combined_outliers),
                'clean_subject_ids': clean_subject_ids
            }
        else:
            print(f"Insufficient clean data for {method} - {metric}")


Debug - POS - MeanNN:
  Total paired subjects: 16
  rPPG values shape: (16,)
  GT values shape: (16,)
  rPPG values: [ 685.0317835   724.78072652  767.45081465  872.12959067  805.06335491
  715.69554609  815.87345618  842.56337236  833.37370739  668.61456841
  809.47333526  747.01954327  808.10152411  747.6761213  1447.31385162
 1128.19599538]
  GT values: [759.2690678  739.79855372 707.57164032 924.78950777 721.64818548
 676.43229167 771.31304825 876.14889706 822.89277523 585.27369281
 831.03197674 779.75543478 856.23504785 667.44402985 707.6951581
 773.77424569]
POS - MeanNN: Removed 2 outliers, kept 14 subjects
Debug - POS - SDNN:
  Total paired subjects: 16
  rPPG values shape: (16,)
  GT values shape: (16,)
  rPPG values: [ 219.68978424  252.56196907  261.86283636  246.67807048  265.90920134
  199.33989862  266.36732233  384.01426937  273.65868381  193.86002914
  178.42975855  228.56326466  232.96396529  261.90142527 2239.63543932
 1725.5345483 ]
  GT values: [256.224354   188.310

In [9]:
## Print the correlation results
for method, metrics in correlation_results.items():
    print(f"Method: {method}")
    for metric, result in metrics.items():
        print(f"  {metric}: Correlation = {result['correlation']:.4f}, p-value = {result['p_value']:.4f}")
    print("\n")

Method: POS
  MeanNN: Correlation = 0.8092, p-value = 0.0005
  SDNN: Correlation = 0.4048, p-value = 0.1700
  RMSSD: Correlation = 0.6198, p-value = 0.0239
  pNN50: Correlation = 0.2064, p-value = 0.4791
  LF: Correlation = -0.3668, p-value = 0.1970
  HF: Correlation = 0.5905, p-value = 0.0205
  LF_HF: Correlation = 0.6974, p-value = 0.0080
  PR: Correlation = 0.6924, p-value = 0.0042


Method: LGI
  MeanNN: Correlation = 0.6838, p-value = 0.0070
  SDNN: Correlation = 0.3253, p-value = 0.2564
  RMSSD: Correlation = 0.2753, p-value = 0.3407
  pNN50: Correlation = 0.2477, p-value = 0.3549
  LF: Correlation = -0.1592, p-value = 0.5866
  HF: Correlation = 0.6838, p-value = 0.0049
  LF_HF: Correlation = 0.5619, p-value = 0.0456
  PR: Correlation = 0.4342, p-value = 0.1208


Method: OMIT
  MeanNN: Correlation = 0.6988, p-value = 0.0054
  SDNN: Correlation = 0.2634, p-value = 0.3628
  RMSSD: Correlation = 0.2243, p-value = 0.4408
  pNN50: Correlation = -0.1455, p-value = 0.6048
  LF: Correlat

In [10]:
def plot_correlation_scatter(rppg_values, gt_values, method, metric, correlation_info=None):
    """ Plot the correlation scatter plot for rPPG values and ground truth values.
    
    Parameters:
    ----------
    rppg_values (list): List of rPPG values.
    gt_values (list): List of ground truth values.
    method (str): The rPPG method used.
    metric (str): The HRV metric being analyzed.
    correlation_info (dict): Dictionary containing correlation statistics.
    """
    plt.figure(figsize=(10, 8))
    
    # Create scatter plot
    sns.scatterplot(x=rppg_values, y=gt_values, s=80, alpha=0.7)
    
    # Add regression line
    sns.regplot(x=rppg_values, y=gt_values, scatter=False, color='red', 
                line_kws={"linewidth": 2, "label": "Regression Line"})
    
    # Add identity line (perfect correlation)
    min_val = min(min(rppg_values), min(gt_values))
    max_val = max(max(rppg_values), max(gt_values))
    plt.plot([min_val, max_val], [min_val, max_val], '--', color='gray', 
             alpha=0.8, linewidth=1, label='Perfect Correlation')
    
    # Set labels and title
    plt.xlabel(f"{method} {metric}", fontsize=12)
    plt.ylabel(f"Ground Truth {metric}", fontsize=12)
    
    # Add correlation statistics to title if available
    if correlation_info:
        corr = correlation_info.get('correlation', 0)
        p_val = correlation_info.get('p_value', 1)
        n_subj = correlation_info.get('n_subjects', len(rppg_values))
        title = f"{method} - {metric}\nr = {corr:.3f}, p = {p_val:.3f}, n = {n_subj}"
    else:
        # Calculate correlation if not provided
        if len(rppg_values) > 1:
            corr, p_val = stats.pearsonr(rppg_values, gt_values)
            title = f"{method} - {metric}\nr = {corr:.3f}, p = {p_val:.3f}, n = {len(rppg_values)}"
        else:
            title = f"{method} - {metric}"
    
    plt.title(title, fontsize=14, fontweight='bold')
    
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# # Plot correlation scatter plots using the cleaned data
# print("\n" + "="*50)
# print("GENERATING CORRELATION PLOTS")
# print("="*50)

# for method in plot_data.keys():
#     for metric in plot_data[method].keys():
#         if metric in plot_data[method] and len(plot_data[method][metric]['rppg_values']) > 1:
#             rppg_vals = plot_data[method][metric]['rppg_values']
#             gt_vals = plot_data[method][metric]['gt_values']
            
#             # Get correlation info if available
#             corr_info = correlation_results.get(method, {}).get(metric, None)
            
#             # Create the plot
#             plot_correlation_scatter(rppg_vals, gt_vals, method, metric, corr_info)



In [11]:
# Calculate the top 5 features with the highest correlation for each rPPG method
top_features = {}
for method, metrics in correlation_results.items():
    sorted_metrics = sorted(metrics.items(), key=lambda x: abs(x[1]['correlation']), reverse=True)
    top_features[method] = sorted_metrics[:5]
print("Top 5 Features with Highest Correlation:")
for method, features in top_features.items():
    print(f"Method: {method}")
    for feature, result in features:
        print(f"  {feature}: Correlation = {result['correlation']:.4f}, p-value = {result['p_value']:.4f}")
    print("\n")
    

Top 5 Features with Highest Correlation:
Method: POS
  MeanNN: Correlation = 0.8092, p-value = 0.0005
  LF_HF: Correlation = 0.6974, p-value = 0.0080
  PR: Correlation = 0.6924, p-value = 0.0042
  RMSSD: Correlation = 0.6198, p-value = 0.0239
  HF: Correlation = 0.5905, p-value = 0.0205


Method: LGI
  MeanNN: Correlation = 0.6838, p-value = 0.0070
  HF: Correlation = 0.6838, p-value = 0.0049
  LF_HF: Correlation = 0.5619, p-value = 0.0456
  PR: Correlation = 0.4342, p-value = 0.1208
  SDNN: Correlation = 0.3253, p-value = 0.2564


Method: OMIT
  MeanNN: Correlation = 0.6988, p-value = 0.0054
  HF: Correlation = 0.4936, p-value = 0.0615
  LF_HF: Correlation = 0.4631, p-value = 0.1110
  PR: Correlation = 0.4427, p-value = 0.1129
  SDNN: Correlation = 0.2634, p-value = 0.3628


Method: GREEN
  PR: Correlation = 0.5199, p-value = 0.0470
  MeanNN: Correlation = 0.4356, p-value = 0.1046
  HF: Correlation = 0.4031, p-value = 0.1363
  LF: Correlation = -0.3835, p-value = 0.1582
  SDNN: Correl

### Check the Bland-Altman, to see the mean bias nad the interlva of the Limit of Aggrement, make sure the point fall within the LoA

In [12]:
# # Check the value of the rPPG and GT with the Bland-Altman plot and 
# # see the measurement agreement between the rPPG methods and the ground truth

# def plot_bland_altman(rppg_values, gt_values, method, metric):
#     """ Plot Bland-Altman plot for rPPG values against ground truth values """
#     mean_diff = np.mean(rppg_values - gt_values)
#     std_diff = np.std(rppg_values - gt_values)

#     plt.figure(figsize=(10, 6))
#     plt.scatter((rppg_values + gt_values) / 2, rppg_values - gt_values, alpha=0.5)
#     plt.axhline(mean_diff, color='red', linestyle='--', label='Mean Difference')
#     plt.axhline(mean_diff + 1.96 * std_diff, color='green', linestyle='--', label='Upper Limit of Agreement')
#     plt.axhline(mean_diff - 1.96 * std_diff, color='blue', linestyle='--', label='Lower Limit of Agreement')
    
#     plt.title(f'Bland-Altman Plot: {method} - {metric}')
#     plt.xlabel('Mean of rPPG and GT Values')
#     plt.ylabel('Difference (rPPG - GT)')
#     plt.legend()
#     plt.grid()
#     plt.show()

# # Plot Bland-Altman plots for each method and metric
# for method in rppg_hrv_metrics.keys():
#     for metric in hrv_metrics.keys():
#         rppg_values = []
#         gt_values = []

#         for subject_id in rppg_hrv_metrics[method].keys():
#             # Use hrv_means for the rPPG values
#             if subject_id in hrv_means[method] and metric in hrv_means[method][subject_id]:
#                 rppg_values.append(hrv_means[method][subject_id][metric])
            
#             # For ground truth, get the first value from the list or calculate mean
#             if subject_id in gt_hrv_metrics and metric in gt_hrv_metrics[subject_id]:
#                 if not gt_hrv_metrics[subject_id][metric].empty:  # Check if the list is not empty
#                     gt_values.append(gt_hrv_metrics[subject_id][metric][0])  # Get first element from list

#         if len(rppg_values) > 0 and len(gt_values) > 0:
#             plot_bland_altman(np.array(rppg_values), np.array(gt_values), method, metric)


In [13]:
def calculate_bland_altman_stats(rppg_values, gt_values):
    """ Calculate the Bland-Altman statistics 
    
    Parameters:
    ----------
    rppg_values (array): rPPG measurement values
    gt_values (array): Ground truth values
    
    Returns:
    --------
    tuple: mean_diff, std_diff, upper_limit, lower_limit, mean_avg
    """
    rppg_values = np.array(rppg_values)
    gt_values = np.array(gt_values)
    
    # Calculate differences and averages
    differences = rppg_values - gt_values
    averages = (rppg_values + gt_values) / 2
    
    mean_diff = np.mean(differences)
    std_diff = np.std(differences, ddof=1)  # Use sample standard deviation
    mean_avg = np.mean(averages)
    
    # Calculate limits of agreement (1.96 * SD)
    upper_limit = mean_diff + 1.96 * std_diff
    lower_limit = mean_diff - 1.96 * std_diff
    
    return mean_diff, std_diff, upper_limit, lower_limit, mean_avg

def calculate_percentage_difference(rppg_values, gt_values):
    """ Calculate the percentage difference between rPPG and ground truth values 
    
    Parameters:
    ----------
    rppg_values (array): rPPG measurement values
    gt_values (array): Ground truth values
    
    Returns:
    --------
    tuple: mean_percentage_diff, median_percentage_diff
    """
    rppg_values = np.array(rppg_values)
    gt_values = np.array(gt_values)
    
    # Avoid division by zero
    mask = gt_values != 0
    if np.sum(mask) == 0:
        return np.nan, np.nan
    
    # Calculate percentage differences
    percentage_diff = np.abs((rppg_values[mask] - gt_values[mask]) / gt_values[mask]) * 100
    
    return np.mean(percentage_diff), np.median(percentage_diff)

def plot_bland_altman(rppg_values, gt_values, method, metric, stats_info=None):
    """ Plot Bland-Altman plot
    
    Parameters:
    ----------
    rppg_values (array): rPPG measurement values
    gt_values (array): Ground truth values
    method (str): Method name
    metric (str): Metric name
    stats_info (dict): Statistics information
    """
    rppg_values = np.array(rppg_values)
    gt_values = np.array(gt_values)
    
    # Calculate differences and averages
    differences = rppg_values - gt_values
    averages = (rppg_values + gt_values) / 2
    
    # Calculate statistics
    mean_diff, std_diff, upper_limit, lower_limit, _ = calculate_bland_altman_stats(rppg_values, gt_values)
    
    # Create the plot
    plt.figure(figsize=(10, 8))
    
    # Scatter plot
    plt.scatter(averages, differences, alpha=0.7, s=60)
    
    # Mean difference line
    plt.axhline(mean_diff, color='red', linestyle='-', linewidth=2, label=f'Mean Diff: {mean_diff:.3f}')
    
    # Limits of agreement
    plt.axhline(upper_limit, color='red', linestyle='--', linewidth=1.5, label=f'Upper LoA: {upper_limit:.3f}')
    plt.axhline(lower_limit, color='red', linestyle='--', linewidth=1.5, label=f'Lower LoA: {lower_limit:.3f}')
    
    # Zero line
    plt.axhline(0, color='black', linestyle='-', alpha=0.3, linewidth=1)
    
    # Labels and title
    plt.xlabel(f'Average of {method} and Ground Truth {metric}', fontsize=12)
    plt.ylabel(f'{method} - Ground Truth {metric}', fontsize=12)
    
    if stats_info:
        n_subj = stats_info.get('n_subjects', len(rppg_values))
        title = f'Bland-Altman Plot: {method} - {metric}\nn = {n_subj}, SD = {std_diff:.3f}'
    else:
        title = f'Bland-Altman Plot: {method} - {metric}\nn = {len(rppg_values)}, SD = {std_diff:.3f}'
    
    plt.title(title, fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


In [14]:
# Calculate Bland-Altman statistics using the cleaned data from correlation analysis
print("\n" + "="*60)
print("CALCULATING BLAND-ALTMAN STATISTICS")
print("="*60)

bland_altman_results = []

for method in plot_data.keys():
    for metric in plot_data[method].keys():
        if metric in plot_data[method] and len(plot_data[method][metric]['rppg_values']) > 1:
            rppg_vals = plot_data[method][metric]['rppg_values']
            gt_vals = plot_data[method][metric]['gt_values']
            n_subjects = len(rppg_vals)
            
            # Calculate Bland-Altman statistics
            mean_diff, std_diff, upper_limit, lower_limit, mean_avg = calculate_bland_altman_stats(rppg_vals, gt_vals)
            
            # Calculate percentage differences
            mean_perc_diff, median_perc_diff = calculate_percentage_difference(rppg_vals, gt_vals)
            
            # Get correlation info if available
            corr_info = correlation_results.get(method, {}).get(metric, {})
            correlation = corr_info.get('correlation', np.nan)
            p_value = corr_info.get('p_value', np.nan)
            
            bland_altman_results.append({
                'Method': method,
                'Metric': metric,
                'N_Subjects': n_subjects,
                'rPPG_Mean': np.mean(rppg_vals),
                'GT_Mean': np.mean(gt_vals),
                'Mean_Difference': mean_diff,
                'Std_Difference': std_diff,
                'Upper_LoA': upper_limit,
                'Lower_LoA': lower_limit,
                'Mean_Percentage_Diff': mean_perc_diff,
                'Median_Percentage_Diff': median_perc_diff,
                'Correlation': correlation,
                'P_Value': p_value
            })

# Convert to DataFrame
bland_altman_df = pd.DataFrame(bland_altman_results)

# Display results with better formatting
print("\nBland-Altman Analysis Results:")
print("-" * 100)

# Create a formatted display
display_df = bland_altman_df.copy()
for col in ['rPPG_Mean', 'GT_Mean', 'Mean_Difference', 'Std_Difference', 
           'Upper_LoA', 'Lower_LoA', 'Mean_Percentage_Diff', 'Median_Percentage_Diff', 
           'Correlation']:
    if col in display_df.columns:
        display_df[col] = display_df[col].apply(lambda x: f"{x:.3f}" if not np.isnan(x) else "N/A")

# Format p-values
if 'P_Value' in display_df.columns:
    display_df['P_Value'] = display_df['P_Value'].apply(lambda x: f"{x:.4f}" if not np.isnan(x) else "N/A")

print(display_df.to_string(index=False))

# Analysis of methods within acceptable limits
print("\n" + "="*60)
print("ANALYSIS OF METHODS WITHIN ACCEPTABLE LIMITS")
print("="*60)

# Methods within 20% difference
within_20_percent = bland_altman_df[bland_altman_df['Mean_Percentage_Diff'] <= 20.0]
print(f"\nMethods within 20% mean percentage difference ({len(within_20_percent)} out of {len(bland_altman_df)}):")
if len(within_20_percent) > 0:
    print(within_20_percent[['Method', 'Metric', 'Mean_Percentage_Diff', 'Correlation', 'N_Subjects']].to_string(index=False))
else:
    print("No methods found within 20% difference threshold.")

# Methods within 10% difference (more stringent)
within_10_percent = bland_altman_df[bland_altman_df['Mean_Percentage_Diff'] <= 10.0]
print(f"\nMethods within 10% mean percentage difference ({len(within_10_percent)} out of {len(bland_altman_df)}):")
if len(within_10_percent) > 0:
    print(within_10_percent[['Method', 'Metric', 'Mean_Percentage_Diff', 'Correlation', 'N_Subjects']].to_string(index=False))
else:
    print("No methods found within 10% difference threshold.")

# Best performing methods (lowest percentage difference)
print(f"\nTop 5 best performing method-metric combinations (lowest mean percentage difference):")
best_methods = bland_altman_df.nsmallest(5, 'Mean_Percentage_Diff')
print(best_methods[['Method', 'Metric', 'Mean_Percentage_Diff', 'Correlation', 'N_Subjects']].to_string(index=False))

# # Generate Bland-Altman plots
# print("\n" + "="*50)
# print("GENERATING BLAND-ALTMAN PLOTS")
# print("="*50)

# for method in plot_data.keys():
#     for metric in plot_data[method].keys():
#         if metric in plot_data[method] and len(plot_data[method][metric]['rppg_values']) > 1:
#             rppg_vals = plot_data[method][metric]['rppg_values']
#             gt_vals = plot_data[method][metric]['gt_values']
            
#             # Get statistics info
#             stats_info = {'n_subjects': len(rppg_vals)}
            
#             # Create Bland-Altman plot
#             plot_bland_altman(rppg_vals, gt_vals, method, metric, stats_info)

# Summary statistics
print("\n" + "="*60)
print("SUMMARY STATISTICS")
print("="*60)

print(f"Total method-metric combinations analyzed: {len(bland_altman_df)}")
print(f"Mean percentage difference across all combinations: {bland_altman_df['Mean_Percentage_Diff'].mean():.2f}%")
print(f"Median percentage difference across all combinations: {bland_altman_df['Mean_Percentage_Diff'].median():.2f}%")
print(f"Best performing combination: {best_methods.iloc[0]['Method']} - {best_methods.iloc[0]['Metric']} ({best_methods.iloc[0]['Mean_Percentage_Diff']:.2f}%)")



CALCULATING BLAND-ALTMAN STATISTICS

Bland-Altman Analysis Results:
----------------------------------------------------------------------------------------------------
Method Metric  N_Subjects rPPG_Mean GT_Mean Mean_Difference Std_Difference Upper_LoA Lower_LoA Mean_Percentage_Diff Median_Percentage_Diff Correlation P_Value
   POS MeanNN          14   774.489 765.686           8.803         55.449   117.483   -99.877                6.635                  5.736       0.809  0.0005
   POS   SDNN          13   237.060 186.074          50.986         61.059   170.663   -68.690               46.674                 33.312       0.405  0.1700
   POS  RMSSD          13   309.427 239.317          70.110         75.774   218.627   -78.407               50.905                 37.066       0.620  0.0239
   POS  pNN50          14    87.104  67.710          19.394         17.799    54.279   -15.491               40.504                 15.019       0.206  0.4791
   POS     LF          14     0.044

### Conclussion : 30 Seconds window

The study correlation within the 30 seconds rppg hrv metrics compare to the GT shows weak / moderate relation with the GT.

Using the bland-altman itself it shows one feature. The MeanNN (time it takes between each heart beat) have acceptable agreement with the reference based on your 20% threshold.

In [15]:
# ## Store the rPPG hrv metrics into the csv
# output_path = "rest_rppg_hrv_metrics_window-30s.csv"

# ## Convert the feature of the CHROM within the HRV Means to be the DataFrame
# #   MeanNN: Correlation = 0.6109, p-value = 0.0119
# #   SD1: Correlation = 0.5190, p-value = 0.0474
# #   RMSSD: Correlation = 0.5185, p-value = 0.0477
# #   LF: Correlation = 0.3975, p-value = 0.1423
# #   SDNN: Correlation = 0.3676, p-value = 0.1959
# ## Take only the CHROM method and the MeanNN, SD1, RMSSD, LF, SDNN
# chrom_hrv_metrics = {
#     'MeanNN': [],
#     'pNN50': [],
#     'RMSSD': [],
#     'SDNN': []
# }

# for subject_id in hrv_means['CHROM'].keys():
#     chrom_hrv_metrics['MeanNN'].append(hrv_means['CHROM'][subject_id]['MeanNN'])
#     chrom_hrv_metrics['pNN50'].append(hrv_means['CHROM'][subject_id]['pNN50'])
#     chrom_hrv_metrics['RMSSD'].append(hrv_means['CHROM'][subject_id]['RMSSD'])
#     chrom_hrv_metrics['SDNN'].append(hrv_means['CHROM'][subject_id]['SDNN'])

# ## Convert the chrom_hrv_metrics to a DataFrame
# chrom_df = pd.DataFrame(chrom_hrv_metrics)

# ## Add label Rest to the dataFrame
# chrom_df['Label'] = 'Rest'

# chrom_df.head()

# ## Save the DataFrame to a CSV file
# chrom_df.to_csv(output_path, index=False)

---

# 1 Minute Plot Correlation

For 1 minute window, the averaging purpose will be done under windowing each short rPPG segment with the **strides** of 30 seconds (means the different between each short window is 30 seconds).

The test will be done under certain scenario of the Task 1, Task 2 UBFC, Physio Rest 2 and Rest 6

In [16]:
root_path = "UBFC-Phys"
subjects = ["s41", "s42", "s43", "s44","s45","s46","s47","s48","s49","s50","s51","s52", "s53","s54","s55","s56"]
tasks = ["T3"]

# Store ground truth and rPPG data
gt_data = {}
rppg_data = {
    'POS': {},
    'LGI': {},
    'OMIT': {},
    'GREEN': {},
    'CHROM': {}
}
# Expected sampling rates (adjust if different for your dataset)
sample_rate_gt = 64  # Hz
sample_rate_video = 35 # Hz


In [17]:
## Process for each subject and task
for subject in subjects:
    for task in tasks:
        subject_task_id = f"{subject}_{task}"

        # Load rPPG signals from different methods
        pos = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_POS_rppg.npy"))
        lgi = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_LGI_rppg.npy"))
        omit = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_OMIT_rppg.npy"))
        green = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_GREEN_rppg.npy"))
        chrom = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_CHROM_rppg.npy"))

        # Load ground truth BVP
        GT = pd.read_csv(os.path.join(root_path, subject, f"bvp_{subject}_{task}.csv")).values
        GT = GT.flatten()

        ## process rPPG signals
        rppg_data["POS"][subject_task_id] = preprocess_ppg(pos, fs=sample_rate_video)
        rppg_data["LGI"][subject_task_id] = preprocess_ppg(lgi, fs=sample_rate_video)
        rppg_data["OMIT"][subject_task_id] = preprocess_ppg(omit, fs=sample_rate_video)
        rppg_data["GREEN"][subject_task_id] = preprocess_ppg(green, fs=sample_rate_video)
        rppg_data["CHROM"][subject_task_id] = preprocess_ppg(chrom, fs=sample_rate_video)
        
        GT = preprocess_ppg(GT, fs=sample_rate_gt)
        gt_data[subject_task_id] = GT

print(f"Done Process the Signals")
    

Done Process the Signals


In [ ]:
"""
Steps to reproduce getting the short term of 30 seconds for each subject + averaging:
1. Loop through each subject.
2. For each short rppg segment (30 seconds), compute the hrv metrics with the neurokit2 package and store it.
3. Average the HRV metrics across all segments for each subject.
4. Compare the correlation between the averaged HRV metrics of the rPPG methods and the ground truth HRV metrics.
# Note: The above code is a preprocessing step. The next steps would involve calculating HRV metrics and performing correlation analysis.
""" 

## Iterate for each subject and compute HRV metrics
hrv_metrics = {
    'MeanNN': [],
    'SDNN': [],
    'RMSSD': [],
    'pNN50': [],
    'LF': [],
    'HF': [],
    'LF_HF': [],
    'SD1': [],
    'SD2': [],
    'PR' : [],
}

## Store the HRV metrics for each rPPG method for each subject
rppg_hrv_metrics = {
    method: {
        subject_id: {
            key: [] for key in hrv_metrics.keys()
        } for subject_id in rppg_data[method].keys()
    } for method in rppg_data.keys()
}

## Iterate through each subject and compute HRV for each segments
for rppg_method in rppg_data.keys():
    for subject_task_id, rppg_signal in rppg_data[rppg_method].items():
        print(f"Processing {subject_task_id} for {rppg_method}")

        ## Applied the window of 30 seconds with stride of 15 seconds
        segment_length = 60 * sample_rate_video
        stride_length = 30 * sample_rate_video
        
        ## Making the segments
        for start in range(0, len(rppg_signal) - segment_length + 1, stride_length):
            segment = rppg_signal[start:start + segment_length]
            ## If the segment is less than the segment length, we skip it
            if len(segment) < segment_length:
                continue

            ## Compute the HRV metrics using neurokit2
            signals, _ = nk.ppg_process(segment, sampling_rate=sample_rate_video)
            peaks, _ = nk.ppg_peaks(signals["PPG_Clean"], sampling_rate=sample_rate_video)

            ## Getting the HR and store it in the metrics
            rppg_hrv_metrics[rppg_method][subject_task_id]['PR'].append(signals['PPG_Rate'][0])

            # Getting the HRV Metrics
            ## Time Domain
            hrv_time = nk.hrv_time(peaks, sampling_rate=sample_rate_video)

            ## Add into the hrv_metrics dictionary
            rppg_hrv_metrics[rppg_method][subject_task_id]['MeanNN'].append(hrv_time['HRV_MeanNN'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['SDNN'].append(hrv_time['HRV_SDNN'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['RMSSD'].append(hrv_time['HRV_RMSSD'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['pNN50'].append(hrv_time['HRV_pNN50'])

            ## Frequency Domain
            hrv_freq = nk.hrv_frequency(peaks, sampling_rate=sample_rate_video, psd_method="welch")
            rppg_hrv_metrics[rppg_method][subject_task_id]['LF'].append(hrv_freq['HRV_LF'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['HF'].append(hrv_freq['HRV_HF'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['LF_HF'].append(hrv_freq['HRV_LFHF'])

            ## Non-Linear Domain
            hrv_non_linear = nk.hrv_nonlinear(peaks, sampling_rate=sample_rate_video)
            rppg_hrv_metrics[rppg_method][subject_task_id]['SD1'].append(hrv_non_linear['HRV_SD1'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['SD2'].append(hrv_non_linear['HRV_SD2'])

Processing s41_T3 for POS
Processing s42_T3 for POS
Processing s43_T3 for POS
Processing s44_T3 for POS
Processing s45_T3 for POS
Processing s46_T3 for POS
Processing s47_T3 for POS
Processing s48_T3 for POS
Processing s49_T3 for POS
Processing s50_T3 for POS
Processing s51_T3 for POS
Processing s52_T3 for POS
Processing s53_T3 for POS
Processing s54_T3 for POS
Processing s55_T3 for POS
Processing s56_T3 for POS
Processing s41_T3 for LGI
Processing s42_T3 for LGI
Processing s43_T3 for LGI
Processing s44_T3 for LGI
Processing s45_T3 for LGI
Processing s46_T3 for LGI
Processing s47_T3 for LGI
Processing s48_T3 for LGI
Processing s49_T3 for LGI
Processing s50_T3 for LGI
Processing s51_T3 for LGI
Processing s52_T3 for LGI
Processing s53_T3 for LGI
Processing s54_T3 for LGI
Processing s55_T3 for LGI
Processing s56_T3 for LGI
Processing s41_T3 for OMIT
Processing s42_T3 for OMIT
Processing s43_T3 for OMIT
Processing s44_T3 for OMIT
Processing s45_T3 for OMIT
Processing s46_T3 for OMIT
Proces

In [19]:
### Calculate the average HRV metrics for each segment for each subject per method

hrv_means = {}
for method in rppg_hrv_metrics:
    hrv_means[method] = {}

    for subject in rppg_hrv_metrics[method]:
        hrv_means[method][subject] = {}

        for metric, values in rppg_hrv_metrics[method][subject].items():
            if values:
                hrv_means[method][subject][metric] = np.mean(values)
            else:
                hrv_means[method][subject][metric] = np.nan

print(hrv_means)

{'POS': {'s41_T3': {'MeanNN': 685.9447004608294, 'SDNN': 217.73005041199366, 'RMSSD': 299.41693318191574, 'pNN50': 86.29032258064517, 'LF': 0.05525054859330854, 'HF': 0.10386730416799139, 'LF_HF': 0.531933980918078, 'SD1': 212.1473523147467, 'SD2': 222.7689569919261, 'PR': 87.47060799462545}, 's42_T3': {'MeanNN': 719.9517781796262, 'SDNN': 245.7080868417813, 'RMSSD': 328.48719838359375, 'pNN50': 86.07594936708861, 'LF': 0.031225175342434075, 'HF': 0.06169732279607503, 'LF_HF': 0.5061025977681565, 'SD1': 232.75590855416422, 'SD2': 255.90511157075832, 'PR': 83.33891493636972}, 's43_T3': {'MeanNN': 757.5255102040817, 'SDNN': 254.04180242735325, 'RMSSD': 311.9885766635028, 'pNN50': 92.85714285714286, 'LF': 0.029258491709946755, 'HF': 0.04982956798571218, 'LF_HF': 0.58717128991237, 'SD1': 221.09294366965634, 'SD2': 280.5583401425671, 'PR': 79.20525340966492}, 's44_T3': {'MeanNN': 867.4927113702623, 'SDNN': 243.75640972716286, 'RMSSD': 312.9040558935957, 'pNN50': 88.26530612244898, 'LF': 0.0

### Getting the GT HRV Metrics

In [20]:
# Compare the Correlation between the averaged HRV metrics of the rPPG methods and the ground truth HRV metrics

## Getting the ground truth HRV metrics

gt_hrv_metrics = {
    subject_id: {
        key: [] for key in hrv_metrics.keys()
    } for subject_id in gt_data.keys()
}

# Iterate through each subject and compute the full length HRV metrics for the ground truth
for subject_task_id, gt_signal in gt_data.items():
    print(f"Processing {subject_task_id} for ground truth")

    ## Compute the HRV metrics using neurokit2
    signals, _ = nk.ppg_process(gt_signal, sampling_rate=sample_rate_gt)
    peaks, _ = nk.ppg_peaks(signals["PPG_Clean"], sampling_rate=sample_rate_gt)

    ## Getting the HR and store it in the metrics
    gt_hrv_metrics[subject_task_id]['PR'] = signals['PPG_Rate'][0].item()
    
    # Getting the HRV Metrics

    ## Time Domain
    hrv_time = nk.hrv_time(peaks, sampling_rate=sample_rate_gt)

    ## Add into the hrv_metrics dictionary
    gt_hrv_metrics[subject_task_id]['MeanNN'] = (hrv_time['HRV_MeanNN'][0])
    gt_hrv_metrics[subject_task_id]['SDNN'] = (hrv_time['HRV_SDNN'][0])
    gt_hrv_metrics[subject_task_id]['RMSSD'] = (hrv_time['HRV_RMSSD'][0])
    gt_hrv_metrics[subject_task_id]['pNN50'] = (hrv_time['HRV_pNN50'][0])

    ## Frequency Domain
    hrv_freq = nk.hrv_frequency(peaks, sampling_rate=sample_rate_gt, psd_method="welch")
    gt_hrv_metrics[subject_task_id]['LF'] = (hrv_freq['HRV_LF'][0])
    gt_hrv_metrics[subject_task_id]['HF'] = (hrv_freq['HRV_HF'][0])
    gt_hrv_metrics[subject_task_id]['LF_HF'] = (hrv_freq['HRV_LFHF'][0])

    ## Non-Linear Domain
    # hrv_non_linear = nk.hrv_nonlinear(peaks, sampling_rate=sample_rate_gt)
    # gt_hrv_metrics[subject_task_id]['SD1'] = (hrv_non_linear['HRV_SD1'])
    # gt_hrv_metrics[subject_task_id]['SD2'] = (hrv_non_linear['HRV_SD2'])

print(gt_hrv_metrics)

Processing s41_T3 for ground truth
Processing s42_T3 for ground truth
Processing s43_T3 for ground truth
Processing s44_T3 for ground truth
Processing s45_T3 for ground truth
Processing s46_T3 for ground truth
Processing s47_T3 for ground truth
Processing s48_T3 for ground truth
Processing s49_T3 for ground truth
Processing s50_T3 for ground truth
Processing s51_T3 for ground truth
Processing s52_T3 for ground truth
Processing s53_T3 for ground truth
Processing s54_T3 for ground truth
Processing s55_T3 for ground truth
Processing s56_T3 for ground truth
{'s41_T3': {'MeanNN': 759.2690677966102, 'SDNN': 256.2243539979767, 'RMSSD': 323.64115992706513, 'pNN50': 78.8135593220339, 'LF': 0.04414474180694006, 'HF': 0.0713327946728041, 'LF_HF': 0.6188561938366115, 'SD1': [], 'SD2': [], 'PR': 79.02336937565399}, 's42_T3': {'MeanNN': 739.7985537190083, 'SDNN': 188.31094883796968, 'RMSSD': 236.80021181113136, 'pNN50': 73.55371900826447, 'LF': 0.031234568707740655, 'HF': 0.07134949016888528, 'LF_HF

### Since we already get the Metrics HRV value of the rPPG, let's compare it with the GT to see the correlation

In [21]:
def identify_outliers_iqr(data):
    """Identify outlier indices using the IQR method.
    
    Parameters:
    ----------
    data (list or numpy array): The data to check for outliers.
    
    Returns:
    --------
    numpy array: Boolean mask where True indicates outlier.
    """
    data = np.asarray(data)
    
    if len(data) == 0:
        return np.array([], dtype=bool)
    
    if len(data) == 1:
        return np.array([False])
    
    # Remove any NaN or infinite values before calculating percentiles
    clean_data = data[np.isfinite(data)]
    
    if len(clean_data) < 2:
        return np.array([False] * len(data))
    
    q1 = np.percentile(clean_data, 25)
    q3 = np.percentile(clean_data, 75)
    iqr = q3 - q1
    
    # Handle case where IQR is 0 (all values are the same)
    if iqr == 0:
        return np.array([False] * len(data))
    
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    outlier_mask = (data < lower_bound) | (data > upper_bound) | ~np.isfinite(data)
    return outlier_mask

# Compute correlation between rPPG methods and ground truth HRV metrics
correlation_results = {}
plot_data = {}  # Store clean data for plotting

for method in hrv_means.keys():
    correlation_results[method] = {}
    plot_data[method] = {}
    
    for metric in hrv_metrics.keys():
        # Collect paired data (subject_id, rppg_value, gt_value)
        paired_data = []
        
        for subject_id in hrv_means[method].keys():
            # Check if both rPPG and GT data exist for this subject and metric
            rppg_available = (subject_id in hrv_means[method] and 
                            metric in hrv_means[method][subject_id])
            gt_available = (subject_id in gt_hrv_metrics and 
                          metric in gt_hrv_metrics[subject_id])
            
            if rppg_available and gt_available:
                rppg_value = hrv_means[method][subject_id][metric]
                gt_value = gt_hrv_metrics[subject_id][metric]
                
                # Handle pandas Series if needed
                if isinstance(gt_value, pd.Series):
                    if not gt_value.empty:
                        gt_value = gt_value.iloc[0]
                    else:
                        continue
                
                # Handle numpy arrays - extract scalar value
                if isinstance(rppg_value, (np.ndarray, list)):
                    if len(rppg_value) > 0:
                        rppg_value = rppg_value[0] if hasattr(rppg_value, '__getitem__') else float(rppg_value)
                    else:
                        continue
                
                if isinstance(gt_value, (np.ndarray, list)):
                    if len(gt_value) > 0:
                        gt_value = gt_value[0] if hasattr(gt_value, '__getitem__') else float(gt_value)
                    else:
                        continue
                
                # Convert to float to ensure scalar values
                try:
                    rppg_value = float(rppg_value)
                    gt_value = float(gt_value)
                except (TypeError, ValueError):
                    print(f"Warning: Could not convert values to float for {subject_id} - {metric}")
                    print(f"  rPPG value type: {type(rppg_value)}, value: {rppg_value}")
                    print(f"  GT value type: {type(gt_value)}, value: {gt_value}")
                    continue
                
                # Check for valid values (now they're guaranteed to be scalars)
                if not np.isnan(rppg_value) and not np.isnan(gt_value):
                    paired_data.append((subject_id, rppg_value, gt_value))
        
        if len(paired_data) < 2:
            print(f"Insufficient data for {method} - {metric}: {len(paired_data)} subjects")
            continue
        
        # Extract values for outlier detection
        subject_ids = [item[0] for item in paired_data]
        rppg_values = np.array([item[1] for item in paired_data])
        gt_values = np.array([item[2] for item in paired_data])
        
        # Debug information
        print(f"Debug - {method} - {metric}:")
        print(f"  Total paired subjects: {len(paired_data)}")
        print(f"  rPPG values shape: {rppg_values.shape}")
        print(f"  GT values shape: {gt_values.shape}")
        print(f"  rPPG values: {rppg_values}")
        print(f"  GT values: {gt_values}")
        
        # Identify outliers in both datasets
        rppg_outliers = identify_outliers_iqr(rppg_values)
        gt_outliers = identify_outliers_iqr(gt_values)
        
        # Combine outlier masks (remove if outlier in either dataset)
        combined_outliers = rppg_outliers | gt_outliers
        
        # Keep only non-outlier subjects
        clean_mask = ~combined_outliers
        clean_rppg_values = rppg_values[clean_mask]
        clean_gt_values = gt_values[clean_mask]
        clean_subject_ids = [subject_ids[i] for i in range(len(subject_ids)) if clean_mask[i]]
        
        print(f"{method} - {metric}: Removed {np.sum(combined_outliers)} outliers, "
              f"kept {len(clean_rppg_values)} subjects")
        
        # Store clean data for plotting
        plot_data[method][metric] = {
            'rppg_values': clean_rppg_values,
            'gt_values': clean_gt_values,
            'subject_ids': clean_subject_ids
        }
        
        # Calculate correlation on clean data
        if len(clean_rppg_values) > 1:
            correlation, p_value = stats.pearsonr(clean_rppg_values, clean_gt_values)
            correlation_results[method][metric] = {
                'correlation': correlation,
                'p_value': p_value,
                'n_subjects': len(clean_rppg_values),
                'removed_subjects': np.sum(combined_outliers),
                'clean_subject_ids': clean_subject_ids
            }
        else:
            print(f"Insufficient clean data for {method} - {metric}")


Debug - POS - MeanNN:
  Total paired subjects: 15
  rPPG values shape: (15,)
  GT values shape: (15,)
  rPPG values: [ 685.94470046  719.95177818  757.5255102   867.49271137  796.7806841
  719.49152542  829.12891986  835.85434174  677.43764172  812.44019139
  752.38095238  808.39539607  738.38509317 1325.34562212 1550.37593985]
  GT values: [759.2690678  739.79855372 707.57164032 924.78950777 721.64818548
 676.43229167 876.14889706 822.89277523 585.27369281 831.03197674
 779.75543478 856.23504785 667.44402985 707.6951581  773.77424569]
POS - MeanNN: Removed 2 outliers, kept 13 subjects
Debug - POS - SDNN:
  Total paired subjects: 15
  rPPG values shape: (15,)
  GT values shape: (15,)
  rPPG values: [ 217.73005041  245.70808684  254.04180243  243.75640973  259.13644523
  192.06278151  284.04217959  278.91087133  201.14198057  171.53588663
  234.87792251  234.16441186  260.2526671  2253.31460155 4154.21786315]
  GT values: [256.224354   188.31094884 166.20538694 218.7653599  119.29290277

In [22]:
## Print the correlation results
for method, metrics in correlation_results.items():
    print(f"Method: {method}")
    for metric, result in metrics.items():
        print(f"  {metric}: Correlation = {result['correlation']:.4f}, p-value = {result['p_value']:.4f}")
    print("\n")

Method: POS
  MeanNN: Correlation = 0.8328, p-value = 0.0004
  SDNN: Correlation = 0.4051, p-value = 0.1915
  RMSSD: Correlation = 0.7439, p-value = 0.0035
  pNN50: Correlation = 0.0285, p-value = 0.9264
  LF: Correlation = 0.3649, p-value = 0.1995
  HF: Correlation = 0.3702, p-value = 0.1744
  LF_HF: Correlation = 0.3109, p-value = 0.3254
  PR: Correlation = 0.8053, p-value = 0.0009


Method: LGI
  MeanNN: Correlation = 0.6985, p-value = 0.0079
  SDNN: Correlation = 0.0298, p-value = 0.9267
  RMSSD: Correlation = 0.1699, p-value = 0.5789
  pNN50: Correlation = 0.0501, p-value = 0.8651
  LF: Correlation = 0.6701, p-value = 0.0063
  HF: Correlation = 0.3801, p-value = 0.1622
  LF_HF: Correlation = 0.3107, p-value = 0.3525
  PR: Correlation = 0.7056, p-value = 0.0070


Method: OMIT
  MeanNN: Correlation = 0.7199, p-value = 0.0055
  SDNN: Correlation = 0.1506, p-value = 0.6403
  RMSSD: Correlation = 0.2326, p-value = 0.4444
  pNN50: Correlation = 0.3201, p-value = 0.2863
  LF: Correlation

In [23]:
def plot_correlation_scatter(rppg_values, gt_values, method, metric, correlation_info=None):
    """ Plot the correlation scatter plot for rPPG values and ground truth values.
    
    Parameters:
    ----------
    rppg_values (list): List of rPPG values.
    gt_values (list): List of ground truth values.
    method (str): The rPPG method used.
    metric (str): The HRV metric being analyzed.
    correlation_info (dict): Dictionary containing correlation statistics.
    """
    plt.figure(figsize=(10, 8))
    
    # Create scatter plot
    sns.scatterplot(x=rppg_values, y=gt_values, s=80, alpha=0.7)
    
    # Add regression line
    sns.regplot(x=rppg_values, y=gt_values, scatter=False, color='red', 
                line_kws={"linewidth": 2, "label": "Regression Line"})
    
    # Add identity line (perfect correlation)
    min_val = min(min(rppg_values), min(gt_values))
    max_val = max(max(rppg_values), max(gt_values))
    plt.plot([min_val, max_val], [min_val, max_val], '--', color='gray', 
             alpha=0.8, linewidth=1, label='Perfect Correlation')
    
    # Set labels and title
    plt.xlabel(f"{method} {metric}", fontsize=12)
    plt.ylabel(f"Ground Truth {metric}", fontsize=12)
    
    # Add correlation statistics to title if available
    if correlation_info:
        corr = correlation_info.get('correlation', 0)
        p_val = correlation_info.get('p_value', 1)
        n_subj = correlation_info.get('n_subjects', len(rppg_values))
        title = f"{method} - {metric}\nr = {corr:.3f}, p = {p_val:.3f}, n = {n_subj}"
    else:
        # Calculate correlation if not provided
        if len(rppg_values) > 1:
            corr, p_val = stats.pearsonr(rppg_values, gt_values)
            title = f"{method} - {metric}\nr = {corr:.3f}, p = {p_val:.3f}, n = {len(rppg_values)}"
        else:
            title = f"{method} - {metric}"
    
    plt.title(title, fontsize=14, fontweight='bold')
    
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# # Plot correlation scatter plots using the cleaned data
# print("\n" + "="*50)
# print("GENERATING CORRELATION PLOTS")
# print("="*50)

# for method in plot_data.keys():
#     for metric in plot_data[method].keys():
#         if metric in plot_data[method] and len(plot_data[method][metric]['rppg_values']) > 1:
#             rppg_vals = plot_data[method][metric]['rppg_values']
#             gt_vals = plot_data[method][metric]['gt_values']
            
#             # Get correlation info if available
#             corr_info = correlation_results.get(method, {}).get(metric, None)
            
#             # Create the plot
#             plot_correlation_scatter(rppg_vals, gt_vals, method, metric, corr_info)



In [24]:
# Calculate the top 5 features with the highest correlation for each rPPG method
top_features = {}
for method, metrics in correlation_results.items():
    sorted_metrics = sorted(metrics.items(), key=lambda x: abs(x[1]['correlation']), reverse=True)
    top_features[method] = sorted_metrics[:5]
print("Top 5 Features with Highest Correlation:")
for method, features in top_features.items():
    print(f"Method: {method}")
    for feature, result in features:
        print(f"  {feature}: Correlation = {result['correlation']:.4f}, p-value = {result['p_value']:.4f}")
    print("\n")
    

Top 5 Features with Highest Correlation:
Method: POS
  MeanNN: Correlation = 0.8328, p-value = 0.0004
  PR: Correlation = 0.8053, p-value = 0.0009
  RMSSD: Correlation = 0.7439, p-value = 0.0035
  SDNN: Correlation = 0.4051, p-value = 0.1915
  HF: Correlation = 0.3702, p-value = 0.1744


Method: LGI
  PR: Correlation = 0.7056, p-value = 0.0070
  MeanNN: Correlation = 0.6985, p-value = 0.0079
  LF: Correlation = 0.6701, p-value = 0.0063
  HF: Correlation = 0.3801, p-value = 0.1622
  LF_HF: Correlation = 0.3107, p-value = 0.3525


Method: OMIT
  PR: Correlation = 0.7371, p-value = 0.0040
  MeanNN: Correlation = 0.7199, p-value = 0.0055
  LF: Correlation = 0.4117, p-value = 0.1273
  LF_HF: Correlation = 0.3240, p-value = 0.3042
  pNN50: Correlation = 0.3201, p-value = 0.2863


Method: GREEN
  HF: Correlation = 0.6700, p-value = 0.0171
  PR: Correlation = 0.6000, p-value = 0.0302
  MeanNN: Correlation = 0.5653, p-value = 0.0441
  LF_HF: Correlation = 0.5336, p-value = 0.0909
  SDNN: Correl

---

### Check the Bland-Altman, to see the mean bias nad the interlva of the Limit of Aggrement, make sure the point fall within the LoA

In [25]:
# # Check the value of the rPPG and GT with the Bland-Altman plot and 
# # see the measurement agreement between the rPPG methods and the ground truth

# def plot_bland_altman(rppg_values, gt_values, method, metric):
#     """ Plot Bland-Altman plot for rPPG values against ground truth values """
#     mean_diff = np.mean(rppg_values - gt_values)
#     std_diff = np.std(rppg_values - gt_values)

#     plt.figure(figsize=(10, 6))
#     plt.scatter((rppg_values + gt_values) / 2, rppg_values - gt_values, alpha=0.5)
#     plt.axhline(mean_diff, color='red', linestyle='--', label='Mean Difference')
#     plt.axhline(mean_diff + 1.96 * std_diff, color='green', linestyle='--', label='Upper Limit of Agreement')
#     plt.axhline(mean_diff - 1.96 * std_diff, color='blue', linestyle='--', label='Lower Limit of Agreement')
    
#     plt.title(f'Bland-Altman Plot: {method} - {metric}')
#     plt.xlabel('Mean of rPPG and GT Values')
#     plt.ylabel('Difference (rPPG - GT)')
#     plt.legend()
#     plt.grid()
#     plt.show()

# # Plot Bland-Altman plots for each method and metric
# for method in rppg_hrv_metrics.keys():
#     for metric in hrv_metrics.keys():
#         rppg_values = []
#         gt_values = []

#         for subject_id in rppg_hrv_metrics[method].keys():
#             # Use hrv_means for the rPPG values
#             if subject_id in hrv_means[method] and metric in hrv_means[method][subject_id]:
#                 rppg_values.append(hrv_means[method][subject_id][metric])
            
#             # For ground truth, get the first value from the list or calculate mean
#             if subject_id in gt_hrv_metrics and metric in gt_hrv_metrics[subject_id]:
#                 if not gt_hrv_metrics[subject_id][metric].empty:  # Check if the list is not empty
#                     gt_values.append(gt_hrv_metrics[subject_id][metric][0])  # Get first element from list

#         if len(rppg_values) > 0 and len(gt_values) > 0:
#             plot_bland_altman(np.array(rppg_values), np.array(gt_values), method, metric)


In [26]:
def calculate_bland_altman_stats(rppg_values, gt_values):
    """ Calculate the Bland-Altman statistics 
    
    Parameters:
    ----------
    rppg_values (array): rPPG measurement values
    gt_values (array): Ground truth values
    
    Returns:
    --------
    tuple: mean_diff, std_diff, upper_limit, lower_limit, mean_avg
    """
    rppg_values = np.array(rppg_values)
    gt_values = np.array(gt_values)
    
    # Calculate differences and averages
    differences = rppg_values - gt_values
    averages = (rppg_values + gt_values) / 2
    
    mean_diff = np.mean(differences)
    std_diff = np.std(differences, ddof=1)  # Use sample standard deviation
    mean_avg = np.mean(averages)
    
    # Calculate limits of agreement (1.96 * SD)
    upper_limit = mean_diff + 1.96 * std_diff
    lower_limit = mean_diff - 1.96 * std_diff
    
    return mean_diff, std_diff, upper_limit, lower_limit, mean_avg

def calculate_percentage_difference(rppg_values, gt_values):
    """ Calculate the percentage difference between rPPG and ground truth values 
    
    Parameters:
    ----------
    rppg_values (array): rPPG measurement values
    gt_values (array): Ground truth values
    
    Returns:
    --------
    tuple: mean_percentage_diff, median_percentage_diff
    """
    rppg_values = np.array(rppg_values)
    gt_values = np.array(gt_values)
    
    # Avoid division by zero
    mask = gt_values != 0
    if np.sum(mask) == 0:
        return np.nan, np.nan
    
    # Calculate percentage differences
    percentage_diff = np.abs((rppg_values[mask] - gt_values[mask]) / gt_values[mask]) * 100
    
    return np.mean(percentage_diff), np.median(percentage_diff)

def plot_bland_altman(rppg_values, gt_values, method, metric, stats_info=None):
    """ Plot Bland-Altman plot
    
    Parameters:
    ----------
    rppg_values (array): rPPG measurement values
    gt_values (array): Ground truth values
    method (str): Method name
    metric (str): Metric name
    stats_info (dict): Statistics information
    """
    rppg_values = np.array(rppg_values)
    gt_values = np.array(gt_values)
    
    # Calculate differences and averages
    differences = rppg_values - gt_values
    averages = (rppg_values + gt_values) / 2
    
    # Calculate statistics
    mean_diff, std_diff, upper_limit, lower_limit, _ = calculate_bland_altman_stats(rppg_values, gt_values)
    
    # Create the plot
    plt.figure(figsize=(10, 8))
    
    # Scatter plot
    plt.scatter(averages, differences, alpha=0.7, s=60)
    
    # Mean difference line
    plt.axhline(mean_diff, color='red', linestyle='-', linewidth=2, label=f'Mean Diff: {mean_diff:.3f}')
    
    # Limits of agreement
    plt.axhline(upper_limit, color='red', linestyle='--', linewidth=1.5, label=f'Upper LoA: {upper_limit:.3f}')
    plt.axhline(lower_limit, color='red', linestyle='--', linewidth=1.5, label=f'Lower LoA: {lower_limit:.3f}')
    
    # Zero line
    plt.axhline(0, color='black', linestyle='-', alpha=0.3, linewidth=1)
    
    # Labels and title
    plt.xlabel(f'Average of {method} and Ground Truth {metric}', fontsize=12)
    plt.ylabel(f'{method} - Ground Truth {metric}', fontsize=12)
    
    if stats_info:
        n_subj = stats_info.get('n_subjects', len(rppg_values))
        title = f'Bland-Altman Plot: {method} - {metric}\nn = {n_subj}, SD = {std_diff:.3f}'
    else:
        title = f'Bland-Altman Plot: {method} - {metric}\nn = {len(rppg_values)}, SD = {std_diff:.3f}'
    
    plt.title(title, fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


In [27]:
# Calculate Bland-Altman statistics using the cleaned data from correlation analysis
print("\n" + "="*60)
print("CALCULATING BLAND-ALTMAN STATISTICS")
print("="*60)

bland_altman_results = []

for method in plot_data.keys():
    for metric in plot_data[method].keys():
        if metric in plot_data[method] and len(plot_data[method][metric]['rppg_values']) > 1:
            rppg_vals = plot_data[method][metric]['rppg_values']
            gt_vals = plot_data[method][metric]['gt_values']
            n_subjects = len(rppg_vals)
            
            # Calculate Bland-Altman statistics
            mean_diff, std_diff, upper_limit, lower_limit, mean_avg = calculate_bland_altman_stats(rppg_vals, gt_vals)
            
            # Calculate percentage differences
            mean_perc_diff, median_perc_diff = calculate_percentage_difference(rppg_vals, gt_vals)
            
            # Get correlation info if available
            corr_info = correlation_results.get(method, {}).get(metric, {})
            correlation = corr_info.get('correlation', np.nan)
            p_value = corr_info.get('p_value', np.nan)
            
            bland_altman_results.append({
                'Method': method,
                'Metric': metric,
                'N_Subjects': n_subjects,
                'rPPG_Mean': np.mean(rppg_vals),
                'GT_Mean': np.mean(gt_vals),
                'Mean_Difference': mean_diff,
                'Std_Difference': std_diff,
                'Upper_LoA': upper_limit,
                'Lower_LoA': lower_limit,
                'Mean_Percentage_Diff': mean_perc_diff,
                'Median_Percentage_Diff': median_perc_diff,
                'Correlation': correlation,
                'P_Value': p_value
            })

# Convert to DataFrame
bland_altman_df = pd.DataFrame(bland_altman_results)

# Display results with better formatting
print("\nBland-Altman Analysis Results:")
print("-" * 100)

# Create a formatted display
display_df = bland_altman_df.copy()
for col in ['rPPG_Mean', 'GT_Mean', 'Mean_Difference', 'Std_Difference', 
           'Upper_LoA', 'Lower_LoA', 'Mean_Percentage_Diff', 'Median_Percentage_Diff', 
           'Correlation']:
    if col in display_df.columns:
        display_df[col] = display_df[col].apply(lambda x: f"{x:.3f}" if not np.isnan(x) else "N/A")

# Format p-values
if 'P_Value' in display_df.columns:
    display_df['P_Value'] = display_df['P_Value'].apply(lambda x: f"{x:.4f}" if not np.isnan(x) else "N/A")

print(display_df.to_string(index=False))

# Analysis of methods within acceptable limits
print("\n" + "="*60)
print("ANALYSIS OF METHODS WITHIN ACCEPTABLE LIMITS")
print("="*60)

# Methods within 20% difference
within_20_percent = bland_altman_df[bland_altman_df['Mean_Percentage_Diff'] <= 20.0]
print(f"\nMethods within 20% mean percentage difference ({len(within_20_percent)} out of {len(bland_altman_df)}):")
if len(within_20_percent) > 0:
    print(within_20_percent[['Method', 'Metric', 'Mean_Percentage_Diff', 'Correlation', 'N_Subjects']].to_string(index=False))
else:
    print("No methods found within 20% difference threshold.")

# Methods within 10% difference (more stringent)
within_10_percent = bland_altman_df[bland_altman_df['Mean_Percentage_Diff'] <= 10.0]
print(f"\nMethods within 10% mean percentage difference ({len(within_10_percent)} out of {len(bland_altman_df)}):")
if len(within_10_percent) > 0:
    print(within_10_percent[['Method', 'Metric', 'Mean_Percentage_Diff', 'Correlation', 'N_Subjects']].to_string(index=False))
else:
    print("No methods found within 10% difference threshold.")

# Best performing methods (lowest percentage difference)
print(f"\nTop 5 best performing method-metric combinations (lowest mean percentage difference):")
best_methods = bland_altman_df.nsmallest(5, 'Mean_Percentage_Diff')
print(best_methods[['Method', 'Metric', 'Mean_Percentage_Diff', 'Correlation', 'N_Subjects']].to_string(index=False))

# # Generate Bland-Altman plots
# print("\n" + "="*50)
# print("GENERATING BLAND-ALTMAN PLOTS")
# print("="*50)

# for method in plot_data.keys():
#     for metric in plot_data[method].keys():
#         if metric in plot_data[method] and len(plot_data[method][metric]['rppg_values']) > 1:
#             rppg_vals = plot_data[method][metric]['rppg_values']
#             gt_vals = plot_data[method][metric]['gt_values']
            
#             # Get statistics info
#             stats_info = {'n_subjects': len(rppg_vals)}
            
#             # Create Bland-Altman plot
#             plot_bland_altman(rppg_vals, gt_vals, method, metric, stats_info)

# Summary statistics
print("\n" + "="*60)
print("SUMMARY STATISTICS")
print("="*60)

print(f"Total method-metric combinations analyzed: {len(bland_altman_df)}")
print(f"Mean percentage difference across all combinations: {bland_altman_df['Mean_Percentage_Diff'].mean():.2f}%")
print(f"Median percentage difference across all combinations: {bland_altman_df['Mean_Percentage_Diff'].median():.2f}%")
print(f"Best performing combination: {best_methods.iloc[0]['Method']} - {best_methods.iloc[0]['Metric']} ({best_methods.iloc[0]['Mean_Percentage_Diff']:.2f}%)")



CALCULATING BLAND-ALTMAN STATISTICS

Bland-Altman Analysis Results:
----------------------------------------------------------------------------------------------------
Method Metric  N_Subjects rPPG_Mean GT_Mean Mean_Difference Std_Difference Upper_LoA Lower_LoA Mean_Percentage_Diff Median_Percentage_Diff Correlation P_Value
   POS MeanNN          13   769.324 765.253           4.071         56.363   114.542  -106.401                6.694                  6.196       0.833  0.0004
   POS   SDNN          12   232.777 179.890          52.886         60.149   170.779   -65.007               48.635                 29.321       0.405  0.1915
   POS  RMSSD          13   310.240 256.357          53.883         99.227   248.368  -140.603               51.850                 33.502       0.744  0.0035
   POS  pNN50          13    87.664  67.445          20.219         19.107    57.668   -17.231               43.360                 17.025       0.028  0.9264
   POS     LF          14     0.041

### Conclussion : 1 Minute Window

Stuff

In [28]:
# ## Store the rPPG hrv metrics into the csv
# output_path = "rest_rppg_hrv_metrics_window-60s.csv"

# ## Convert the feature of the CHROM within the HRV Means to be the DataFrame
# #   MeanNN: Correlation = 0.6109, p-value = 0.0119
# #   SD1: Correlation = 0.5190, p-value = 0.0474
# #   RMSSD: Correlation = 0.5185, p-value = 0.0477
# #   LF: Correlation = 0.3975, p-value = 0.1423
# #   SDNN: Correlation = 0.3676, p-value = 0.1959
# ## Take only the CHROM method and the MeanNN, SD1, RMSSD, LF, SDNN
# chrom_hrv_metrics = {
#     'MeanNN': [],
#     'pNN50': [],
#     'RMSSD': [],
#     'SDNN': []
# }

# for subject_id in hrv_means['CHROM'].keys():
#     chrom_hrv_metrics['MeanNN'].append(hrv_means['CHROM'][subject_id]['MeanNN'])
#     chrom_hrv_metrics['pNN50'].append(hrv_means['CHROM'][subject_id]['pNN50'])
#     chrom_hrv_metrics['RMSSD'].append(hrv_means['CHROM'][subject_id]['RMSSD'])
#     chrom_hrv_metrics['SDNN'].append(hrv_means['CHROM'][subject_id]['SDNN'])

# ## Convert the chrom_hrv_metrics to a DataFrame
# chrom_df = pd.DataFrame(chrom_hrv_metrics)

# ## Add label Rest to the dataFrame
# chrom_df['Label'] = 'Rest'

# chrom_df.head()

# ## Save the DataFrame to a CSV file
# chrom_df.to_csv(output_path, index=False)

---

# 2 Minute Plot Correlation

For 2 minute window, the averaging purpose will be done under windowing each short rPPG segment with the **strides** of 60 seconds (means the different between each short window is 60 seconds).

The test will be done under certain scenario of the Task 1, Task 2 UBFC, Physio Rest 2 and Rest 6

In [29]:
root_path = "UBFC-Phys"
subjects = ["s41", "s42", "s43", "s44","s45","s46","s47","s48","s49","s50","s51","s52", "s53","s54","s55","s56"]
tasks = ["T3"]

# Store ground truth and rPPG data
gt_data = {}
rppg_data = {
    'POS': {},
    'LGI': {},
    'OMIT': {},
    'GREEN': {},
    'CHROM': {}
}
# Expected sampling rates (adjust if different for your dataset)
sample_rate_gt = 64  # Hz
sample_rate_video = 35 # Hz


In [30]:
## Process for each subject and task
for subject in subjects:
    for task in tasks:
        subject_task_id = f"{subject}_{task}"

        # Load rPPG signals from different methods
        pos = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_POS_rppg.npy"))
        lgi = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_LGI_rppg.npy"))
        omit = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_OMIT_rppg.npy"))
        green = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_GREEN_rppg.npy"))
        chrom = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_CHROM_rppg.npy"))

        # Load ground truth BVP
        GT = pd.read_csv(os.path.join(root_path, subject, f"bvp_{subject}_{task}.csv")).values
        GT = GT.flatten()

        ## process rPPG signals
        rppg_data["POS"][subject_task_id] = preprocess_ppg(pos, fs=sample_rate_video)
        rppg_data["LGI"][subject_task_id] = preprocess_ppg(lgi, fs=sample_rate_video)
        rppg_data["OMIT"][subject_task_id] = preprocess_ppg(omit, fs=sample_rate_video)
        rppg_data["GREEN"][subject_task_id] = preprocess_ppg(green, fs=sample_rate_video)
        rppg_data["CHROM"][subject_task_id] = preprocess_ppg(chrom, fs=sample_rate_video)
        
        GT = preprocess_ppg(GT, fs=sample_rate_gt)
        gt_data[subject_task_id] = GT

print(f"Done Process the Signals")
    

Done Process the Signals


In [ ]:
"""
Steps to reproduce getting the short term of 30 seconds for each subject + averaging:
1. Loop through each subject.
2. For each short rppg segment (30 seconds), compute the hrv metrics with the neurokit2 package and store it.
3. Average the HRV metrics across all segments for each subject.
4. Compare the correlation between the averaged HRV metrics of the rPPG methods and the ground truth HRV metrics.
# Note: The above code is a preprocessing step. The next steps would involve calculating HRV metrics and performing correlation analysis.
""" 

## Iterate for each subject and compute HRV metrics
hrv_metrics = {
    'MeanNN': [],
    'SDNN': [],
    'RMSSD': [],
    'pNN50': [],
    'LF': [],
    'HF': [],
    'LF_HF': [],
    'SD1': [],
    'SD2': [],
    'PR' : [],
}

## Store the HRV metrics for each rPPG method for each subject
rppg_hrv_metrics = {
    method: {
        subject_id: {
            key: [] for key in hrv_metrics.keys()
        } for subject_id in rppg_data[method].keys()
    } for method in rppg_data.keys()
}

## Iterate through each subject and compute HRV for each segments
for rppg_method in rppg_data.keys():
    for subject_task_id, rppg_signal in rppg_data[rppg_method].items():
        print(f"Processing {subject_task_id} for {rppg_method}")

        ## Applied the window of 30 seconds with stride of 15 seconds
        segment_length = 120 * sample_rate_video
        stride_length = 45 * sample_rate_video
        
        ## Making the segments
        for start in range(0, len(rppg_signal) - segment_length + 1, stride_length):
            segment = rppg_signal[start:start + segment_length]
            ## If the segment is less than the segment length, we skip it
            if len(segment) < segment_length:
                continue

            ## Compute the HRV metrics using neurokit2
            signals, _ = nk.ppg_process(segment, sampling_rate=sample_rate_video)
            peaks, _ = nk.ppg_peaks(signals["PPG_Clean"], sampling_rate=sample_rate_video)

            ## Getting the HR and store it in the metrics
            rppg_hrv_metrics[rppg_method][subject_task_id]['PR'].append(signals['PPG_Rate'][0])

            # Getting the HRV Metrics
            ## Time Domain
            hrv_time = nk.hrv_time(peaks, sampling_rate=sample_rate_video)

            ## Add into the hrv_metrics dictionary
            rppg_hrv_metrics[rppg_method][subject_task_id]['MeanNN'].append(hrv_time['HRV_MeanNN'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['SDNN'].append(hrv_time['HRV_SDNN'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['RMSSD'].append(hrv_time['HRV_RMSSD'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['pNN50'].append(hrv_time['HRV_pNN50'])

            ## Frequency Domain
            hrv_freq = nk.hrv_frequency(peaks, sampling_rate=sample_rate_video, psd_method="welch")
            rppg_hrv_metrics[rppg_method][subject_task_id]['LF'].append(hrv_freq['HRV_LF'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['HF'].append(hrv_freq['HRV_HF'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['LF_HF'].append(hrv_freq['HRV_LFHF'])

            ## Non-Linear Domain
            hrv_non_linear = nk.hrv_nonlinear(peaks, sampling_rate=sample_rate_video)
            rppg_hrv_metrics[rppg_method][subject_task_id]['SD1'].append(hrv_non_linear['HRV_SD1'])
            rppg_hrv_metrics[rppg_method][subject_task_id]['SD2'].append(hrv_non_linear['HRV_SD2'])

Processing s41_T3 for POS
Processing s42_T3 for POS
Processing s43_T3 for POS
Processing s44_T3 for POS
Processing s45_T3 for POS
Processing s46_T3 for POS
Processing s47_T3 for POS
Processing s48_T3 for POS
Processing s49_T3 for POS
Processing s50_T3 for POS
Processing s51_T3 for POS
Processing s52_T3 for POS
Processing s53_T3 for POS
Processing s54_T3 for POS
Processing s55_T3 for POS
Processing s56_T3 for POS
Processing s41_T3 for LGI
Processing s42_T3 for LGI
Processing s43_T3 for LGI
Processing s44_T3 for LGI
Processing s45_T3 for LGI
Processing s46_T3 for LGI
Processing s47_T3 for LGI
Processing s48_T3 for LGI
Processing s49_T3 for LGI
Processing s50_T3 for LGI
Processing s51_T3 for LGI
Processing s52_T3 for LGI
Processing s53_T3 for LGI
Processing s54_T3 for LGI
Processing s55_T3 for LGI
Processing s56_T3 for LGI
Processing s41_T3 for OMIT
Processing s42_T3 for OMIT
Processing s43_T3 for OMIT
Processing s44_T3 for OMIT
Processing s45_T3 for OMIT
Processing s46_T3 for OMIT
Proces

In [32]:
### Calculate the average HRV metrics for each segment for each subject per method

hrv_means = {}
for method in rppg_hrv_metrics:
    hrv_means[method] = {}

    for subject in rppg_hrv_metrics[method]:
        hrv_means[method][subject] = {}

        for metric, values in rppg_hrv_metrics[method][subject].items():
            if values:
                hrv_means[method][subject][metric] = np.mean(values)
            else:
                hrv_means[method][subject][metric] = np.nan

print(hrv_means)

{'POS': {'s41_T3': {'MeanNN': 685.9447004608294, 'SDNN': 217.73005041199366, 'RMSSD': 299.41693318191574, 'pNN50': 86.29032258064517, 'LF': 0.05525054859330854, 'HF': 0.10386730416799139, 'LF_HF': 0.531933980918078, 'SD1': 212.1473523147467, 'SD2': 222.7689569919261, 'PR': 87.47060799462545}, 's42_T3': {'MeanNN': 719.9517781796262, 'SDNN': 245.7080868417813, 'RMSSD': 328.48719838359375, 'pNN50': 86.07594936708861, 'LF': 0.031225175342434075, 'HF': 0.06169732279607503, 'LF_HF': 0.5061025977681565, 'SD1': 232.75590855416422, 'SD2': 255.90511157075832, 'PR': 83.33891493636972}, 's43_T3': {'MeanNN': 757.5255102040817, 'SDNN': 254.04180242735325, 'RMSSD': 311.9885766635028, 'pNN50': 92.85714285714286, 'LF': 0.029258491709946755, 'HF': 0.04982956798571218, 'LF_HF': 0.58717128991237, 'SD1': 221.09294366965634, 'SD2': 280.5583401425671, 'PR': 79.20525340966492}, 's44_T3': {'MeanNN': 867.4927113702623, 'SDNN': 243.75640972716286, 'RMSSD': 312.9040558935957, 'pNN50': 88.26530612244898, 'LF': 0.0

### Getting the GT HRV Metrics

In [33]:
# Compare the Correlation between the averaged HRV metrics of the rPPG methods and the ground truth HRV metrics

## Getting the ground truth HRV metrics

gt_hrv_metrics = {
    subject_id: {
        key: [] for key in hrv_metrics.keys()
    } for subject_id in gt_data.keys()
}

# Iterate through each subject and compute the full length HRV metrics for the ground truth
for subject_task_id, gt_signal in gt_data.items():
    print(f"Processing {subject_task_id} for ground truth")

    ## Compute the HRV metrics using neurokit2
    signals, _ = nk.ppg_process(gt_signal, sampling_rate=sample_rate_gt)
    peaks, _ = nk.ppg_peaks(signals["PPG_Clean"], sampling_rate=sample_rate_gt)

    ## Getting the HR and store it in the metrics
    gt_hrv_metrics[subject_task_id]['PR'] = signals['PPG_Rate'][0].item()
    
    # Getting the HRV Metrics

    ## Time Domain
    hrv_time = nk.hrv_time(peaks, sampling_rate=sample_rate_gt)

    ## Add into the hrv_metrics dictionary
    gt_hrv_metrics[subject_task_id]['MeanNN'] = (hrv_time['HRV_MeanNN'][0])
    gt_hrv_metrics[subject_task_id]['SDNN'] = (hrv_time['HRV_SDNN'][0])
    gt_hrv_metrics[subject_task_id]['RMSSD'] = (hrv_time['HRV_RMSSD'][0])
    gt_hrv_metrics[subject_task_id]['pNN50'] = (hrv_time['HRV_pNN50'][0])

    ## Frequency Domain
    hrv_freq = nk.hrv_frequency(peaks, sampling_rate=sample_rate_gt, psd_method="welch")
    gt_hrv_metrics[subject_task_id]['LF'] = (hrv_freq['HRV_LF'][0])
    gt_hrv_metrics[subject_task_id]['HF'] = (hrv_freq['HRV_HF'][0])
    gt_hrv_metrics[subject_task_id]['LF_HF'] = (hrv_freq['HRV_LFHF'][0])

    ## Non-Linear Domain
    # hrv_non_linear = nk.hrv_nonlinear(peaks, sampling_rate=sample_rate_gt)
    # gt_hrv_metrics[subject_task_id]['SD1'] = (hrv_non_linear['HRV_SD1'])
    # gt_hrv_metrics[subject_task_id]['SD2'] = (hrv_non_linear['HRV_SD2'])

print(gt_hrv_metrics)

Processing s41_T3 for ground truth
Processing s42_T3 for ground truth
Processing s43_T3 for ground truth
Processing s44_T3 for ground truth
Processing s45_T3 for ground truth
Processing s46_T3 for ground truth
Processing s47_T3 for ground truth
Processing s48_T3 for ground truth
Processing s49_T3 for ground truth
Processing s50_T3 for ground truth
Processing s51_T3 for ground truth
Processing s52_T3 for ground truth
Processing s53_T3 for ground truth
Processing s54_T3 for ground truth
Processing s55_T3 for ground truth
Processing s56_T3 for ground truth
{'s41_T3': {'MeanNN': 759.2690677966102, 'SDNN': 256.2243539979767, 'RMSSD': 323.64115992706513, 'pNN50': 78.8135593220339, 'LF': 0.04414474180694006, 'HF': 0.0713327946728041, 'LF_HF': 0.6188561938366115, 'SD1': [], 'SD2': [], 'PR': 79.02336937565399}, 's42_T3': {'MeanNN': 739.7985537190083, 'SDNN': 188.31094883796968, 'RMSSD': 236.80021181113136, 'pNN50': 73.55371900826447, 'LF': 0.031234568707740655, 'HF': 0.07134949016888528, 'LF_HF

### Since we already get the Metrics HRV value of the rPPG, let's compare it with the GT to see the correlation

In [34]:
def identify_outliers_iqr(data):
    """Identify outlier indices using the IQR method.
    
    Parameters:
    ----------
    data (list or numpy array): The data to check for outliers.
    
    Returns:
    --------
    numpy array: Boolean mask where True indicates outlier.
    """
    data = np.asarray(data)
    
    if len(data) == 0:
        return np.array([], dtype=bool)
    
    if len(data) == 1:
        return np.array([False])
    
    # Remove any NaN or infinite values before calculating percentiles
    clean_data = data[np.isfinite(data)]
    
    if len(clean_data) < 2:
        return np.array([False] * len(data))
    
    q1 = np.percentile(clean_data, 25)
    q3 = np.percentile(clean_data, 75)
    iqr = q3 - q1
    
    # Handle case where IQR is 0 (all values are the same)
    if iqr == 0:
        return np.array([False] * len(data))
    
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    outlier_mask = (data < lower_bound) | (data > upper_bound) | ~np.isfinite(data)
    return outlier_mask

# Compute correlation between rPPG methods and ground truth HRV metrics
correlation_results = {}
plot_data = {}  # Store clean data for plotting

for method in hrv_means.keys():
    correlation_results[method] = {}
    plot_data[method] = {}
    
    for metric in hrv_metrics.keys():
        # Collect paired data (subject_id, rppg_value, gt_value)
        paired_data = []
        
        for subject_id in hrv_means[method].keys():
            # Check if both rPPG and GT data exist for this subject and metric
            rppg_available = (subject_id in hrv_means[method] and 
                            metric in hrv_means[method][subject_id])
            gt_available = (subject_id in gt_hrv_metrics and 
                          metric in gt_hrv_metrics[subject_id])
            
            if rppg_available and gt_available:
                rppg_value = hrv_means[method][subject_id][metric]
                gt_value = gt_hrv_metrics[subject_id][metric]
                
                # Handle pandas Series if needed
                if isinstance(gt_value, pd.Series):
                    if not gt_value.empty:
                        gt_value = gt_value.iloc[0]
                    else:
                        continue
                
                # Handle numpy arrays - extract scalar value
                if isinstance(rppg_value, (np.ndarray, list)):
                    if len(rppg_value) > 0:
                        rppg_value = rppg_value[0] if hasattr(rppg_value, '__getitem__') else float(rppg_value)
                    else:
                        continue
                
                if isinstance(gt_value, (np.ndarray, list)):
                    if len(gt_value) > 0:
                        gt_value = gt_value[0] if hasattr(gt_value, '__getitem__') else float(gt_value)
                    else:
                        continue
                
                # Convert to float to ensure scalar values
                try:
                    rppg_value = float(rppg_value)
                    gt_value = float(gt_value)
                except (TypeError, ValueError):
                    print(f"Warning: Could not convert values to float for {subject_id} - {metric}")
                    print(f"  rPPG value type: {type(rppg_value)}, value: {rppg_value}")
                    print(f"  GT value type: {type(gt_value)}, value: {gt_value}")
                    continue
                
                # Check for valid values (now they're guaranteed to be scalars)
                if not np.isnan(rppg_value) and not np.isnan(gt_value):
                    paired_data.append((subject_id, rppg_value, gt_value))
        
        if len(paired_data) < 2:
            print(f"Insufficient data for {method} - {metric}: {len(paired_data)} subjects")
            continue
        
        # Extract values for outlier detection
        subject_ids = [item[0] for item in paired_data]
        rppg_values = np.array([item[1] for item in paired_data])
        gt_values = np.array([item[2] for item in paired_data])
        
        # Debug information
        print(f"Debug - {method} - {metric}:")
        print(f"  Total paired subjects: {len(paired_data)}")
        print(f"  rPPG values shape: {rppg_values.shape}")
        print(f"  GT values shape: {gt_values.shape}")
        print(f"  rPPG values: {rppg_values}")
        print(f"  GT values: {gt_values}")
        
        # Identify outliers in both datasets
        rppg_outliers = identify_outliers_iqr(rppg_values)
        gt_outliers = identify_outliers_iqr(gt_values)
        
        # Combine outlier masks (remove if outlier in either dataset)
        combined_outliers = rppg_outliers | gt_outliers
        
        # Keep only non-outlier subjects
        clean_mask = ~combined_outliers
        clean_rppg_values = rppg_values[clean_mask]
        clean_gt_values = gt_values[clean_mask]
        clean_subject_ids = [subject_ids[i] for i in range(len(subject_ids)) if clean_mask[i]]
        
        print(f"{method} - {metric}: Removed {np.sum(combined_outliers)} outliers, "
              f"kept {len(clean_rppg_values)} subjects")
        
        # Store clean data for plotting
        plot_data[method][metric] = {
            'rppg_values': clean_rppg_values,
            'gt_values': clean_gt_values,
            'subject_ids': clean_subject_ids
        }
        
        # Calculate correlation on clean data
        if len(clean_rppg_values) > 1:
            correlation, p_value = stats.pearsonr(clean_rppg_values, clean_gt_values)
            correlation_results[method][metric] = {
                'correlation': correlation,
                'p_value': p_value,
                'n_subjects': len(clean_rppg_values),
                'removed_subjects': np.sum(combined_outliers),
                'clean_subject_ids': clean_subject_ids
            }
        else:
            print(f"Insufficient clean data for {method} - {metric}")


Debug - POS - MeanNN:
  Total paired subjects: 15
  rPPG values shape: (15,)
  GT values shape: (15,)
  rPPG values: [ 685.94470046  719.95177818  757.5255102   867.49271137  796.7806841
  719.49152542  829.12891986  835.85434174  677.43764172  812.44019139
  752.38095238  808.39539607  738.38509317 1325.34562212 1550.37593985]
  GT values: [759.2690678  739.79855372 707.57164032 924.78950777 721.64818548
 676.43229167 876.14889706 822.89277523 585.27369281 831.03197674
 779.75543478 856.23504785 667.44402985 707.6951581  773.77424569]
POS - MeanNN: Removed 2 outliers, kept 13 subjects
Debug - POS - SDNN:
  Total paired subjects: 15
  rPPG values shape: (15,)
  GT values shape: (15,)
  rPPG values: [ 217.73005041  245.70808684  254.04180243  243.75640973  259.13644523
  192.06278151  284.04217959  278.91087133  201.14198057  171.53588663
  234.87792251  234.16441186  260.2526671  2253.31460155 4154.21786315]
  GT values: [256.224354   188.31094884 166.20538694 218.7653599  119.29290277

In [35]:
## Print the correlation results
for method, metrics in correlation_results.items():
    print(f"Method: {method}")
    for metric, result in metrics.items():
        print(f"  {metric}: Correlation = {result['correlation']:.4f}, p-value = {result['p_value']:.4f}")
    print("\n")

Method: POS
  MeanNN: Correlation = 0.8328, p-value = 0.0004
  SDNN: Correlation = 0.4051, p-value = 0.1915
  RMSSD: Correlation = 0.7439, p-value = 0.0035
  pNN50: Correlation = 0.0285, p-value = 0.9264
  LF: Correlation = 0.3649, p-value = 0.1995
  HF: Correlation = 0.3702, p-value = 0.1744
  LF_HF: Correlation = 0.3109, p-value = 0.3254
  PR: Correlation = 0.8053, p-value = 0.0009


Method: LGI
  MeanNN: Correlation = 0.6985, p-value = 0.0079
  SDNN: Correlation = 0.0298, p-value = 0.9267
  RMSSD: Correlation = 0.1699, p-value = 0.5789
  pNN50: Correlation = 0.0501, p-value = 0.8651
  LF: Correlation = 0.6701, p-value = 0.0063
  HF: Correlation = 0.3801, p-value = 0.1622
  LF_HF: Correlation = 0.3107, p-value = 0.3525
  PR: Correlation = 0.7056, p-value = 0.0070


Method: OMIT
  MeanNN: Correlation = 0.7199, p-value = 0.0055
  SDNN: Correlation = 0.1506, p-value = 0.6403
  RMSSD: Correlation = 0.2326, p-value = 0.4444
  pNN50: Correlation = 0.3201, p-value = 0.2863
  LF: Correlation

In [36]:
def plot_correlation_scatter(rppg_values, gt_values, method, metric, correlation_info=None):
    """ Plot the correlation scatter plot for rPPG values and ground truth values.
    
    Parameters:
    ----------
    rppg_values (list): List of rPPG values.
    gt_values (list): List of ground truth values.
    method (str): The rPPG method used.
    metric (str): The HRV metric being analyzed.
    correlation_info (dict): Dictionary containing correlation statistics.
    """
    plt.figure(figsize=(10, 8))
    
    # Create scatter plot
    sns.scatterplot(x=rppg_values, y=gt_values, s=80, alpha=0.7)
    
    # Add regression line
    sns.regplot(x=rppg_values, y=gt_values, scatter=False, color='red', 
                line_kws={"linewidth": 2, "label": "Regression Line"})
    
    # Add identity line (perfect correlation)
    min_val = min(min(rppg_values), min(gt_values))
    max_val = max(max(rppg_values), max(gt_values))
    plt.plot([min_val, max_val], [min_val, max_val], '--', color='gray', 
             alpha=0.8, linewidth=1, label='Perfect Correlation')
    
    # Set labels and title
    plt.xlabel(f"{method} {metric}", fontsize=12)
    plt.ylabel(f"Ground Truth {metric}", fontsize=12)
    
    # Add correlation statistics to title if available
    if correlation_info:
        corr = correlation_info.get('correlation', 0)
        p_val = correlation_info.get('p_value', 1)
        n_subj = correlation_info.get('n_subjects', len(rppg_values))
        title = f"{method} - {metric}\nr = {corr:.3f}, p = {p_val:.3f}, n = {n_subj}"
    else:
        # Calculate correlation if not provided
        if len(rppg_values) > 1:
            corr, p_val = stats.pearsonr(rppg_values, gt_values)
            title = f"{method} - {metric}\nr = {corr:.3f}, p = {p_val:.3f}, n = {len(rppg_values)}"
        else:
            title = f"{method} - {metric}"
    
    plt.title(title, fontsize=14, fontweight='bold')
    
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# # Plot correlation scatter plots using the cleaned data
# print("\n" + "="*50)
# print("GENERATING CORRELATION PLOTS")
# print("="*50)

# for method in plot_data.keys():
#     for metric in plot_data[method].keys():
#         if metric in plot_data[method] and len(plot_data[method][metric]['rppg_values']) > 1:
#             rppg_vals = plot_data[method][metric]['rppg_values']
#             gt_vals = plot_data[method][metric]['gt_values']
            
#             # Get correlation info if available
#             corr_info = correlation_results.get(method, {}).get(metric, None)
            
#             # Create the plot
#             plot_correlation_scatter(rppg_vals, gt_vals, method, metric, corr_info)



In [37]:
# Calculate the top 5 features with the highest correlation for each rPPG method
top_features = {}
for method, metrics in correlation_results.items():
    sorted_metrics = sorted(metrics.items(), key=lambda x: abs(x[1]['correlation']), reverse=True)
    top_features[method] = sorted_metrics[:5]
print("Top 5 Features with Highest Correlation:")
for method, features in top_features.items():
    print(f"Method: {method}")
    for feature, result in features:
        print(f"  {feature}: Correlation = {result['correlation']:.4f}, p-value = {result['p_value']:.4f}")
    print("\n")
    

Top 5 Features with Highest Correlation:
Method: POS
  MeanNN: Correlation = 0.8328, p-value = 0.0004
  PR: Correlation = 0.8053, p-value = 0.0009
  RMSSD: Correlation = 0.7439, p-value = 0.0035
  SDNN: Correlation = 0.4051, p-value = 0.1915
  HF: Correlation = 0.3702, p-value = 0.1744


Method: LGI
  PR: Correlation = 0.7056, p-value = 0.0070
  MeanNN: Correlation = 0.6985, p-value = 0.0079
  LF: Correlation = 0.6701, p-value = 0.0063
  HF: Correlation = 0.3801, p-value = 0.1622
  LF_HF: Correlation = 0.3107, p-value = 0.3525


Method: OMIT
  PR: Correlation = 0.7371, p-value = 0.0040
  MeanNN: Correlation = 0.7199, p-value = 0.0055
  LF: Correlation = 0.4117, p-value = 0.1273
  LF_HF: Correlation = 0.3240, p-value = 0.3042
  pNN50: Correlation = 0.3201, p-value = 0.2863


Method: GREEN
  HF: Correlation = 0.6700, p-value = 0.0171
  PR: Correlation = 0.6000, p-value = 0.0302
  MeanNN: Correlation = 0.5653, p-value = 0.0441
  LF_HF: Correlation = 0.5336, p-value = 0.0909
  SDNN: Correl

### Check the Bland-Altman, to see the mean bias nad the interlva of the Limit of Aggrement, make sure the point fall within the LoA

In [38]:
# # Check the value of the rPPG and GT with the Bland-Altman plot and 
# # see the measurement agreement between the rPPG methods and the ground truth

# def plot_bland_altman(rppg_values, gt_values, method, metric):
#     """ Plot Bland-Altman plot for rPPG values against ground truth values """
#     mean_diff = np.mean(rppg_values - gt_values)
#     std_diff = np.std(rppg_values - gt_values)

#     plt.figure(figsize=(10, 6))
#     plt.scatter((rppg_values + gt_values) / 2, rppg_values - gt_values, alpha=0.5)
#     plt.axhline(mean_diff, color='red', linestyle='--', label='Mean Difference')
#     plt.axhline(mean_diff + 1.96 * std_diff, color='green', linestyle='--', label='Upper Limit of Agreement')
#     plt.axhline(mean_diff - 1.96 * std_diff, color='blue', linestyle='--', label='Lower Limit of Agreement')
    
#     plt.title(f'Bland-Altman Plot: {method} - {metric}')
#     plt.xlabel('Mean of rPPG and GT Values')
#     plt.ylabel('Difference (rPPG - GT)')
#     plt.legend()
#     plt.grid()
#     plt.show()

# # Plot Bland-Altman plots for each method and metric
# for method in rppg_hrv_metrics.keys():
#     for metric in hrv_metrics.keys():
#         rppg_values = []
#         gt_values = []

#         for subject_id in rppg_hrv_metrics[method].keys():
#             # Use hrv_means for the rPPG values
#             if subject_id in hrv_means[method] and metric in hrv_means[method][subject_id]:
#                 rppg_values.append(hrv_means[method][subject_id][metric])
            
#             # For ground truth, get the first value from the list or calculate mean
#             if subject_id in gt_hrv_metrics and metric in gt_hrv_metrics[subject_id]:
#                 if not gt_hrv_metrics[subject_id][metric].empty:  # Check if the list is not empty
#                     gt_values.append(gt_hrv_metrics[subject_id][metric][0])  # Get first element from list

#         if len(rppg_values) > 0 and len(gt_values) > 0:
#             plot_bland_altman(np.array(rppg_values), np.array(gt_values), method, metric)


In [39]:
def calculate_bland_altman_stats(rppg_values, gt_values):
    """ Calculate the Bland-Altman statistics 
    
    Parameters:
    ----------
    rppg_values (array): rPPG measurement values
    gt_values (array): Ground truth values
    
    Returns:
    --------
    tuple: mean_diff, std_diff, upper_limit, lower_limit, mean_avg
    """
    rppg_values = np.array(rppg_values)
    gt_values = np.array(gt_values)
    
    # Calculate differences and averages
    differences = rppg_values - gt_values
    averages = (rppg_values + gt_values) / 2
    
    mean_diff = np.mean(differences)
    std_diff = np.std(differences, ddof=1)  # Use sample standard deviation
    mean_avg = np.mean(averages)
    
    # Calculate limits of agreement (1.96 * SD)
    upper_limit = mean_diff + 1.96 * std_diff
    lower_limit = mean_diff - 1.96 * std_diff
    
    return mean_diff, std_diff, upper_limit, lower_limit, mean_avg

def calculate_percentage_difference(rppg_values, gt_values):
    """ Calculate the percentage difference between rPPG and ground truth values 
    
    Parameters:
    ----------
    rppg_values (array): rPPG measurement values
    gt_values (array): Ground truth values
    
    Returns:
    --------
    tuple: mean_percentage_diff, median_percentage_diff
    """
    rppg_values = np.array(rppg_values)
    gt_values = np.array(gt_values)
    
    # Avoid division by zero
    mask = gt_values != 0
    if np.sum(mask) == 0:
        return np.nan, np.nan
    
    # Calculate percentage differences
    percentage_diff = np.abs((rppg_values[mask] - gt_values[mask]) / gt_values[mask]) * 100
    
    return np.mean(percentage_diff), np.median(percentage_diff)

def plot_bland_altman(rppg_values, gt_values, method, metric, stats_info=None):
    """ Plot Bland-Altman plot
    
    Parameters:
    ----------
    rppg_values (array): rPPG measurement values
    gt_values (array): Ground truth values
    method (str): Method name
    metric (str): Metric name
    stats_info (dict): Statistics information
    """
    rppg_values = np.array(rppg_values)
    gt_values = np.array(gt_values)
    
    # Calculate differences and averages
    differences = rppg_values - gt_values
    averages = (rppg_values + gt_values) / 2
    
    # Calculate statistics
    mean_diff, std_diff, upper_limit, lower_limit, _ = calculate_bland_altman_stats(rppg_values, gt_values)
    
    # Create the plot
    plt.figure(figsize=(10, 8))
    
    # Scatter plot
    plt.scatter(averages, differences, alpha=0.7, s=60)
    
    # Mean difference line
    plt.axhline(mean_diff, color='red', linestyle='-', linewidth=2, label=f'Mean Diff: {mean_diff:.3f}')
    
    # Limits of agreement
    plt.axhline(upper_limit, color='red', linestyle='--', linewidth=1.5, label=f'Upper LoA: {upper_limit:.3f}')
    plt.axhline(lower_limit, color='red', linestyle='--', linewidth=1.5, label=f'Lower LoA: {lower_limit:.3f}')
    
    # Zero line
    plt.axhline(0, color='black', linestyle='-', alpha=0.3, linewidth=1)
    
    # Labels and title
    plt.xlabel(f'Average of {method} and Ground Truth {metric}', fontsize=12)
    plt.ylabel(f'{method} - Ground Truth {metric}', fontsize=12)
    
    if stats_info:
        n_subj = stats_info.get('n_subjects', len(rppg_values))
        title = f'Bland-Altman Plot: {method} - {metric}\nn = {n_subj}, SD = {std_diff:.3f}'
    else:
        title = f'Bland-Altman Plot: {method} - {metric}\nn = {len(rppg_values)}, SD = {std_diff:.3f}'
    
    plt.title(title, fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


In [40]:
# Calculate Bland-Altman statistics using the cleaned data from correlation analysis
print("\n" + "="*60)
print("CALCULATING BLAND-ALTMAN STATISTICS")
print("="*60)

bland_altman_results = []

for method in plot_data.keys():
    for metric in plot_data[method].keys():
        if metric in plot_data[method] and len(plot_data[method][metric]['rppg_values']) > 1:
            rppg_vals = plot_data[method][metric]['rppg_values']
            gt_vals = plot_data[method][metric]['gt_values']
            n_subjects = len(rppg_vals)
            
            # Calculate Bland-Altman statistics
            mean_diff, std_diff, upper_limit, lower_limit, mean_avg = calculate_bland_altman_stats(rppg_vals, gt_vals)
            
            # Calculate percentage differences
            mean_perc_diff, median_perc_diff = calculate_percentage_difference(rppg_vals, gt_vals)
            
            # Get correlation info if available
            corr_info = correlation_results.get(method, {}).get(metric, {})
            correlation = corr_info.get('correlation', np.nan)
            p_value = corr_info.get('p_value', np.nan)
            
            bland_altman_results.append({
                'Method': method,
                'Metric': metric,
                'N_Subjects': n_subjects,
                'rPPG_Mean': np.mean(rppg_vals),
                'GT_Mean': np.mean(gt_vals),
                'Mean_Difference': mean_diff,
                'Std_Difference': std_diff,
                'Upper_LoA': upper_limit,
                'Lower_LoA': lower_limit,
                'Mean_Percentage_Diff': mean_perc_diff,
                'Median_Percentage_Diff': median_perc_diff,
                'Correlation': correlation,
                'P_Value': p_value
            })

# Convert to DataFrame
bland_altman_df = pd.DataFrame(bland_altman_results)

# Display results with better formatting
print("\nBland-Altman Analysis Results:")
print("-" * 100)

# Create a formatted display
display_df = bland_altman_df.copy()
for col in ['rPPG_Mean', 'GT_Mean', 'Mean_Difference', 'Std_Difference', 
           'Upper_LoA', 'Lower_LoA', 'Mean_Percentage_Diff', 'Median_Percentage_Diff', 
           'Correlation']:
    if col in display_df.columns:
        display_df[col] = display_df[col].apply(lambda x: f"{x:.3f}" if not np.isnan(x) else "N/A")

# Format p-values
if 'P_Value' in display_df.columns:
    display_df['P_Value'] = display_df['P_Value'].apply(lambda x: f"{x:.4f}" if not np.isnan(x) else "N/A")

print(display_df.to_string(index=False))

# Analysis of methods within acceptable limits
print("\n" + "="*60)
print("ANALYSIS OF METHODS WITHIN ACCEPTABLE LIMITS")
print("="*60)

# Methods within 20% difference
within_20_percent = bland_altman_df[bland_altman_df['Mean_Percentage_Diff'] <= 20.0]
print(f"\nMethods within 20% mean percentage difference ({len(within_20_percent)} out of {len(bland_altman_df)}):")
if len(within_20_percent) > 0:
    print(within_20_percent[['Method', 'Metric', 'Mean_Percentage_Diff', 'Correlation', 'N_Subjects']].to_string(index=False))
else:
    print("No methods found within 20% difference threshold.")

# Methods within 10% difference (more stringent)
within_10_percent = bland_altman_df[bland_altman_df['Mean_Percentage_Diff'] <= 10.0]
print(f"\nMethods within 10% mean percentage difference ({len(within_10_percent)} out of {len(bland_altman_df)}):")
if len(within_10_percent) > 0:
    print(within_10_percent[['Method', 'Metric', 'Mean_Percentage_Diff', 'Correlation', 'N_Subjects']].to_string(index=False))
else:
    print("No methods found within 10% difference threshold.")

# Best performing methods (lowest percentage difference)
print(f"\nTop 5 best performing method-metric combinations (lowest mean percentage difference):")
best_methods = bland_altman_df.nsmallest(5, 'Mean_Percentage_Diff')
print(best_methods[['Method', 'Metric', 'Mean_Percentage_Diff', 'Correlation', 'N_Subjects']].to_string(index=False))

# # Generate Bland-Altman plots
# print("\n" + "="*50)
# print("GENERATING BLAND-ALTMAN PLOTS")
# print("="*50)

# for method in plot_data.keys():
#     for metric in plot_data[method].keys():
#         if metric in plot_data[method] and len(plot_data[method][metric]['rppg_values']) > 1:
#             rppg_vals = plot_data[method][metric]['rppg_values']
#             gt_vals = plot_data[method][metric]['gt_values']
            
#             # Get statistics info
#             stats_info = {'n_subjects': len(rppg_vals)}
            
#             # Create Bland-Altman plot
#             plot_bland_altman(rppg_vals, gt_vals, method, metric, stats_info)

# Summary statistics
print("\n" + "="*60)
print("SUMMARY STATISTICS")
print("="*60)

print(f"Total method-metric combinations analyzed: {len(bland_altman_df)}")
print(f"Mean percentage difference across all combinations: {bland_altman_df['Mean_Percentage_Diff'].mean():.2f}%")
print(f"Median percentage difference across all combinations: {bland_altman_df['Mean_Percentage_Diff'].median():.2f}%")
print(f"Best performing combination: {best_methods.iloc[0]['Method']} - {best_methods.iloc[0]['Metric']} ({best_methods.iloc[0]['Mean_Percentage_Diff']:.2f}%)")



CALCULATING BLAND-ALTMAN STATISTICS

Bland-Altman Analysis Results:
----------------------------------------------------------------------------------------------------
Method Metric  N_Subjects rPPG_Mean GT_Mean Mean_Difference Std_Difference Upper_LoA Lower_LoA Mean_Percentage_Diff Median_Percentage_Diff Correlation P_Value
   POS MeanNN          13   769.324 765.253           4.071         56.363   114.542  -106.401                6.694                  6.196       0.833  0.0004
   POS   SDNN          12   232.777 179.890          52.886         60.149   170.779   -65.007               48.635                 29.321       0.405  0.1915
   POS  RMSSD          13   310.240 256.357          53.883         99.227   248.368  -140.603               51.850                 33.502       0.744  0.0035
   POS  pNN50          13    87.664  67.445          20.219         19.107    57.668   -17.231               43.360                 17.025       0.028  0.9264
   POS     LF          14     0.041

### Conclussion : 2 Minute Window

Stuff

In [41]:
# ## Store the rPPG hrv metrics into the csv
# output_path = "rest_rppg_hrv_metrics_window-120s.csv"

# ## Convert the feature of the CHROM within the HRV Means to be the DataFrame
# #   MeanNN: Correlation = 0.6109, p-value = 0.0119
# #   SD1: Correlation = 0.5190, p-value = 0.0474
# #   RMSSD: Correlation = 0.5185, p-value = 0.0477
# #   LF: Correlation = 0.3975, p-value = 0.1423
# #   SDNN: Correlation = 0.3676, p-value = 0.1959
# ## Take only the CHROM method and the MeanNN, SD1, RMSSD, LF, SDNN
# chrom_hrv_metrics = {
#     'MeanNN': [],
#     'pNN50': [],
#     'RMSSD': [],
#     'SDNN': []
# }

# for subject_id in hrv_means['CHROM'].keys():
#     chrom_hrv_metrics['MeanNN'].append(hrv_means['CHROM'][subject_id]['MeanNN'])
#     chrom_hrv_metrics['pNN50'].append(hrv_means['CHROM'][subject_id]['pNN50'])
#     chrom_hrv_metrics['RMSSD'].append(hrv_means['CHROM'][subject_id]['RMSSD'])
#     chrom_hrv_metrics['SDNN'].append(hrv_means['CHROM'][subject_id]['SDNN'])

# ## Convert the chrom_hrv_metrics to a DataFrame
# chrom_df = pd.DataFrame(chrom_hrv_metrics)

# ## Add label Rest to the dataFrame
# chrom_df['Label'] = 'Rest'

# chrom_df.head()

# ## Save the DataFrame to a CSV file
# chrom_df.to_csv(output_path, index=False)